In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import random
import re
from itertools import chain,repeat,count,islice
from collections import Counter
import itertools

In [1]:
###################################### Dicionarios ############################################

# estes dicionarios servem para dar um peso a cada carta no badugi, ou seja o rei tem mais peso que o As por exemplo
dic_values_to_real_value={
    'A':1,
    '2':2,
    '3':3,
    '4':4,
    '5':5,
    '6':6,
    '7':7,
    '8':8,
    '9':9,
    '10':10,
    'D':12,
    'V':11,
    'R':13
}


dic_real_value_to_values={
     1:'A',
     2:'2',
     3:'3',
     4:'4',
     5:'5',
     6:'6',
     7:'7',
     8:'8',
     9:'9',
     10:'10',
     11:'V',
     12:'D',
     13:'R'
}


################################################################### Funcoes auxiliares ##########################################################
#################################################################################################################################################



##################################################### retira_valores e retira_naipe ################################################################
def retira_valores(mao):
  valores=[]
  for i in range(len(mao)):
    if(len(mao[i])<=2):
      valores.append(mao[i][0])
    else:
      valores.append('10')
  return valores

#valores=retira_valores(mao)
#print("valores",valores)


# a retira naipes extrai os naipes das cartas da mão e retorna uma lista

def retira_naipe(mao):
  naipe=[]
  for i in range(len(mao)):
    if(len(mao[i])<=2):
      naipe.append(mao[i][1])
    else:
      naipe.append(mao[i][2])
  return naipe




############################################################# retira_n_cartas ############################################################################################
def retira_n_cartas (baralho,n):
  baralho_aux=baralho
  if(n>len(baralho_aux)):
    return [[],baralho_aux]
    print("inválido, não se pode escolher mais cartas do que as que existem")
  else:
    if (n==0):
      return [[],baralho_aux]
    else:
      
      t=len(baralho_aux)
      mao=[]
      c=1
      while (c):
        dist=tfd.Categorical(probs=[1/t]*t)
        samples=dist.sample(n)
        samples_array=samples.numpy().tolist()
        aux=sorted(set(samples_array))
        if (len(aux)==n):
          c=0
        #print('debugg:',samples_array)
      for i in range(n):
        mao.append(baralho_aux[samples_array[i]])
      #print('debugcards',mao)
      for j in range(n):
        #print(mao[j])
        baralho.remove(mao[j])
      
      return[mao,baralho]





################################################## RemoverCartas ##########################################################
def removerCartas(baralhoCompleto, baralhoARemover): #result= cartas do baralhoCompleto sem cartas do baralhoARemover
	
	result= baralhoCompleto

	for carta in baralhoARemover:
			result.remove(carta)

	return result


################################################ Combinacoes sem repeticoes ################################################

#retorna todas as combinacoes de uma mao sem repeticoes
       
def combinacoesSemRepeticoes(r, mao):
    [a,b,c,d]= mao
    if (r==1):
        return [(a,),(b,),(c,),(d,)]
    elif (r==2):
        return [(a,b),(a,c),(a,d),(b,c),(b,d),(c,d)] #AB, AC, AD, BC, BD, CD
    elif (r==3):
        return [(a,b,c),(a,b,d),(a,c,d),(b,c,d)] #ABC, ABD, ACD, BCD
    elif (r==4):
        return [(a,b,c,d)]

############################################ Funcao retorna_dic_1 #########################################

# esta função pega numa carta e retorna o valor, serviu para usar no key do sort de forma a ordenar a mão por ordem crescente
def retorna_dic_1(valor_da_mao):
  if(len(valor_da_mao)<=2):
    return dic_values_to_real_value[valor_da_mao[0]]
  else:
    return dic_values_to_real_value['10']

########################################## Funcao retorna_valor_dic #######################################

def retorna_valor_dic(mao):
  mao_aux=mao
  for i in range(len(mao)):
    if(len(mao[i])<=2):
      mao[i]=str(dic_values_to_real_value[mao[i][0]])+mao[i][1]
    else:
      mao[i]=str(dic_values_to_real_value['10'])+mao[i][2]
  return mao



############################################ Funcao ValoresDaMao #########################################

#esta função retorna a nossa mao apenas com os valores e nao com os naipes
#Exemplo: mao = mao = ['2C','7C','RO','5E'] => listaValores = ['2', '7', '13', '5']

def ValoresDaMao(mao):

  CartasComValores = retorna_valor_dic(mao)
  #Cuidado que agora mao = ['2', '7', '13', '5']
  listaValores = []
  for carta in CartasComValores:
    if(len(carta)==2):
      #nao estamos no caso do 10,11,12
      x=carta[0]
      listaValores.append(x)
    else:
      x=carta[0]
      y=carta[1]
      z=x+y
      listaValores.append(z)
  
  #retorna apenas os valores da mao
  return listaValores


############################################# Funcao HaElementosIguais_SN ###########################################

#Esta função diz-nos se temos elementos iguais, por exemplo, se tivermos ['2E','2O','RC','10C'] => a funcao vai retornar 1 porque temos duas vezes o 2 
def HaElementosIguais_SN(mao):
  
  repetidos = 0

  #apenas vamos pegar nos valores da mao, nao nos interessa o naipe
  listaValoresMao = ValoresDaMao(mao)

  #vamos obter a lista sem repeticoes
  listaSemRepet = list(dict.fromkeys(listaValoresMao))

  if(len(listaValoresMao)!=len(listaSemRepet)):
    #há repetidos
    repetidos = 1
  
  return repetidos

############################################## Funcao retira_naipe #####################################################

#Recebe um mao e retira os naipes da mao

def retira_naipe(mao):
  naipe=[]
  for i in range(len(mao)):
    if(len(mao[i])<=2):
      naipe.append(mao[i][1])
    else:
      naipe.append(mao[i][2])
  return naipe


############################################## Funcao HaNaipessIguais_SN ###########################################

# esta Função diz-nos se há naipes repetidos. Se tem naipes repetidos retorna 1

def HaNaipesIguais_SN(mao):
  
  repetidos = 0

  listaNaipesMao = retira_naipe(mao)

  listaSemRepet = list(dict.fromkeys(listaNaipesMao))

  if(len(listaNaipesMao)!=len(listaSemRepet)):
    #há repetidos
    repetidos = 1
  
  return repetidos

############################################# Funcao SomaValoresMao ###########################################

def SomaValoresMao(maoValores):

  sum = 0
  for i in range(len(maoValores)):
    sum += int(maoValores[i])

  return sum

############################################# Funcao retorna_MaoComNumerosELetras ###############################

# Funcao recebe ['1E','2C','13O'] e da ['AE', '2C', 'RO']

def retorna_MaoComNumerosELetras(mao):

  for i in range(len(mao)):
    if(len(mao[i])==2):
      #numero de 1 elemento(1,2,3,4,5...,9)
      mao[i]= dic_real_value_to_values[int(mao[i][0])] + mao[i][1]
    else:
      mao[i]= dic_real_value_to_values[int(mao[i][0]+mao[i][1])] + mao[i][2]
  
  return mao


############################################## Funcao ConjuntoComMenorMao ######################################

def ConjuntoComMenorMao(PossibleResults):

  listaSomas = []

  if(len(PossibleResults)==1):
    # entao esta é a unica solução
    return PossibleResults[0]
  
  else:
    #é porque temos mais do que uma solução. Agora temos de escolher a menor mao

    # MenorMaoDoConjunto recebe uma lista de maos e retorna a menor mao
    menorMao = compara(PossibleResults)

    #Da-nos a mao correspondente
    maoEquivalente = retorna_MaoComNumerosELetras(menorMao)

    return maoEquivalente 

########################################################Funcao Melhor mao(2maos)############################################################################

def retorna_dic_1_aux(valor_da_mao):
  if(valor_da_mao=='10'):
    return dic_values_to_real_value['10']
  if(len(valor_da_mao)<=2):
    return dic_values_to_real_value[valor_da_mao[0]]

def auxilia(n):
  return list(map(retorna_dic_1_aux,n))


def desempat(n):#n é um array de maos de 4 cartas cada
  #assumindo que ja recebe as cartas traduzidas para "pontos"
  #[[1,3,4,5],[1,2,3,4],[4,5,6,7]]
  # tem que ter o maior elemento
  #assumindo 4 cartas por conjunto
  #-----------------------------------caso em que tenho maos iguais-------------------------------------------------------
  aux = []
  n_2 = list(map(sorted,n))
  n2_o = len(n_2[0])
  menor=20
  lista=[]
  j= n2_o-1
  while(j!=0):
    if(n_2[0][j]<n_2[1][j]):
      return n_2[0]
    elif(n_2[1][j]<n_2[0][j]):
      return n_2[1]
    elif(n_2[0][j]==n_2[1][j]):
      j = j-1
  return 0

#print(desempat([[1,2,4],[1,4,2]]))


def desempate_verd(n):
  aux=[]
  aux2=[]
  for i in range(len(n)):
    #print(i)
    #print(len(n))
    c=0
    for j in range(len(n)):
      #print(j)
      #print(len(n))
      if(desempat([n[i],n[j]])==sorted(n[i])):
        c=c+1
        aux=[]
        aux.append(n[i])
      elif(desempat([n[i],n[j]])==sorted(n[j])):
        aux=[]
        #aux.append(n[j])
      elif(desempat([n[i],n[j]])==0):
        c=c+1
        aux=[]
        aux.append(n[i])
    aux2.append([n[i],c])
  maior=0
  s=[]
  for k in aux2:
    if(k[1]>maior):
      maior=k[1]
      s=k[0]
    
  return s

#desempate_verd([[1,2,5],[1,4,2],[8,9,10],[4,5,6]])

def inteiros(n):
  return list(map(int,n))
 
def compara_varios(maos):
  #[["AC","5O",...]]
  valores = list(map(retira_valores,maos)) #vou ter os valores da mao em strings
  naipes = list(map(retira_naipe,maos)) #vou ter os naipes das maos em string
  #tenho de passar os valores a "pontos" com a função converte(alterar depois) e dá me em strings
  pontuacao = list(map(auxilia,valores))
  pontuacao_inteiros = list(map(inteiros,pontuacao))
  melhor_mao = desempat(pontuacao_inteiros)
  #print(melhor_mao)
  if(melhor_mao==0):
    return 0
  c=0
  for i in range(len(pontuacao_inteiros)):
    if(sorted(pontuacao_inteiros[i])==melhor_mao):
      c=i
  #print(c)
  return maos[c]

#################################################################### melhor mao com varias maos ###########################################################

def retorna_dic_1_aux(valor_da_mao):
  if(valor_da_mao=='10'):
    return dic_values_to_real_value['10']
  if(len(valor_da_mao)<=2):
    return dic_values_to_real_value[valor_da_mao[0]]

def auxilia(n):
  return list(map(retorna_dic_1_aux,n))


def desempat(n):#n é um array de maos de 4 cartas cada
  #assumindo que ja recebe as cartas traduzidas para "pontos"
  #[[1,3,4,5],[1,2,3,4],[4,5,6,7]]
  # tem que ter o maior elemento
  #assumindo 4 cartas por conjunto
  #-----------------------------------caso em que tenho maos iguais-------------------------------------------------------
  aux = []
  n_2 = list(map(sorted,n))
  n2_o = len(n_2[0])
  menor=20
  lista=[]
  j= n2_o-1
  while(j!=0):
    if(n_2[0][j]<n_2[1][j]):
      return n_2[0]
    elif(n_2[1][j]<n_2[0][j]):
      return n_2[1]
    elif(n_2[0][j]==n_2[1][j]):
      j = j-1
  return 0

#print(desempat([[1,2,4],[1,4,2]]))


def desempate_verd(n):
  aux=[]
  aux2=[]
  for i in range(len(n)):
    #print(i)
    #print(len(n))
    c=0
    for j in range(len(n)):
      #print(j)
      #print(len(n))
      if(desempat([n[i],n[j]])==sorted(n[i])):
        c=c+1
        aux=[]
        aux.append(n[i])
      elif(desempat([n[i],n[j]])==sorted(n[j])):
        aux=[]
        #aux.append(n[j])
      elif(desempat([n[i],n[j]])==0):
        c=c+1
        aux=[]
        aux.append(n[i])
    aux2.append([n[i],c])
  maior=0
  s=[]
  for k in aux2:
    if(k[1]>maior):
      maior=k[1]
      s=k[0]
    
  return s

#desempate_verd([[1,2,5],[1,4,2],[8,9,10],[4,5,6]])

def inteiros(n):
  return list(map(int,n))
 

############################################### Função compara ##########################################################

#pega em varias maos e da-nos o conjunto menor

def compara(maos):
  #[["AC","5O",...]]
  valores = list(map(retira_valores,maos)) #vou ter os valores da mao em strings
  naipes = list(map(retira_naipe,maos)) #vou ter os naipes das maos em string
  #tenho de passar os valores a "pontos" com a função converte(alterar depois) e dá me em strings
  pontuacao = list(map(auxilia,valores))
  pontuacao_inteiros = list(map(inteiros,pontuacao))
  melhor_mao = desempate_verd(pontuacao_inteiros)
  #print(melhor_mao)
  c=0
  for i in range(len(pontuacao_inteiros)):
    if(sorted(pontuacao_inteiros[i])==melhor_mao):
      c=i
  #print(c)
  return maos[c]
################################################################### FIM DAS Funcoes auxiliares ##########################################################
#########################################################################################################################################################





############################################################################################################################
################################################## Função MaoEquivalente ####################################################
#############################################################################################################################

def MaoEquivalente(mao):

  #rank da mao
  rank = 0

  #lista com todas as possibilidades
  PossibleResults = []

  #Combinações de 4 cartas - 1 combinacao
  combinacoes4cartas = combinacoesSemRepeticoes(4, mao)
  #Como len(combinacoes4cartas) = 1 => só temos de chamar as funçoes que ve valores repetidos e naipes iguais

  ElementosIguaisSN = HaElementosIguais_SN(list(combinacoes4cartas[0]))
  NaipesIguaisSN = HaNaipesIguais_SN(list(combinacoes4cartas[0]))

  if(ElementosIguaisSN | NaipesIguaisSN):
    # se entramos aqui ou há elementos iguais ou naipes iguais ou os dois => nao queremos o 'combinacao'
    x = 1
  else:
    # se entramos aqui nao ha elementos iguais nem naipes iguais
    PossibleResults.append(list(combinacoes4cartas[0]))
    rank = 4

  #Combinaçõe de 3 cartas - 4 combinacoes
  combinacoes3cartas = combinacoesSemRepeticoes(3, mao)

  
  if(PossibleResults==[]):
    for combinacao in combinacoes3cartas:
      ElementosIguaisSN = HaElementosIguais_SN(list(combinacao))
      NaipesIguaisSN = HaNaipesIguais_SN(list(combinacao))

      if(ElementosIguaisSN | NaipesIguaisSN):
        # se entramos aqui ou há elementos iguais ou naipes iguais ou os dois => nao queremos o 'combinacao'
        x=1
      else:
        # se entramos aqui nao ha elementos iguais nem naipes iguais
        PossibleResults.append(list(combinacao))
        rank = 3


  #Combinaçõe de 2 cartas - 6 combinacoes
  combinacoes2cartas = combinacoesSemRepeticoes(2, mao)


  if(PossibleResults==[]):
    for combinacao in combinacoes2cartas:
      ElementosIguaisSN = HaElementosIguais_SN(list(combinacao))
      NaipesIguaisSN = HaNaipesIguais_SN(list(combinacao))

      if(ElementosIguaisSN | NaipesIguaisSN):
        # se entramos aqui ou há elementos iguais ou naipes iguais ou os dois => nao queremos o 'combinacao'
        x=1
      else:
        # se entramos aqui nao ha elementos iguais nem naipes iguais
        PossibleResults.append(list(combinacao))
        rank =2

  #Combinaçõe de 1 cartas - 4 combinacoes
  combinacoes1cartas = combinacoesSemRepeticoes(1, mao)

  if(PossibleResults==[]):
    for combinacao in combinacoes1cartas:
      ElementosIguaisSN = HaElementosIguais_SN(list(combinacao))
      NaipesIguaisSN = HaNaipesIguais_SN(list(combinacao))

      if(ElementosIguaisSN | NaipesIguaisSN):
        # se entramos aqui ou há elementos iguais ou naipes iguais ou os dois => nao queremos o 'combinacao'
        x=1
      else:
        # se entramos aqui nao ha elementos iguais nem naipes iguais
        PossibleResults.append(list(combinacao))
        rank = 1


  #print(PossibleResults)
  # 2C 3C   2C 5P
  #Agora vamos selecionar o conjunto correto, ou seja, o conjunto menor
  #Se tivermos por exemplo: ['6C', '8C', 'RP', 'RO'] podemos ter como solucoes ['6C', 'RO'] ou ['6C', 'RP'] e o pc escolhe uma delas porque aqui nao faz diferença
  maoEquivalente = compara(PossibleResults)

  return [rank,maoEquivalente]





###########################################################################################################################
################################################# Funcao decideDraw #######################################################
###########################################################################################################################

def decideDraw_1carta(maoPC_ordenada, cartasPossiveis):
	
	#Descartando apenas 1 carta, qual devemos descartar (se é que devemos descartar alguma)?

	#maoPC_ordenada=[A,B,C,D] # mão atual do PC por ordem crescente de rank

	#novaMaoA = [aleatória, B, C, D]

	#cartasPossiveis = [cartas que podem substituir a carta que estamos a descartar]

	#nMudarMaoN= numero de cartas possiveis que mudam o tipo de mão para melhor, ao descartar a carta N
	#nMelhorarN= numero de cartas possiveis que mudam a mão para melhor, no geral (seja melhorando o tipo de mão, seja reduzindo a carta maior), ao descartar a carta N
	
	nMudarMaoA= 0 
	nMelhorarA= 0 

	nMudarMaoB= 0 
	nMelhorarB= 0

	nMudarMaoC= 0 
	nMelhorarC= 0

	nMudarMaoD= 0 
	nMelhorarD= 0
	

	for cartaNova in cartasPossiveis:
		
		#novaMaoA= [cartaNova, B, C, D] = possível nova mão descartando a A
		#novaMaoB= [A, cartaNova, C, D] = possível nova mão descartando a B
		#novaMaoC= [A, B, cartaNova, D] = possível nova mão descartando a C
		#novaMaoD= [A, B, C, cartaNova] = possível nova mão descartando a D

		cartaA= maoPC_ordenada[0]
		cartaB= maoPC_ordenada[1]
		cartaC= maoPC_ordenada[2]
		cartaD= maoPC_ordenada[3]

		novaMaoA= maoPC_ordenada[:]
		novaMaoA.remove(cartaA)
		novaMaoA.append(cartaNova)

		novaMaoB= maoPC_ordenada[:]
		novaMaoB.remove(cartaB)
		novaMaoB.append(cartaNova)

		novaMaoC= maoPC_ordenada[:]
		novaMaoC.remove(cartaC)
		novaMaoC.append(cartaNova)

		novaMaoD= maoPC_ordenada[:]
		novaMaoD.remove(cartaD)
		novaMaoD.append(cartaNova)

		# Investigar probabilidades para o caso de descartarmos a carta A:

		categoriaMaoPC_ordenada= MaoEquivalente(maoPC_ordenada)[0]
		categoria_novaMaoA= MaoEquivalente(novaMaoA)[0]
		
		if (categoriaMaoPC_ordenada < categoria_novaMaoA):
			
			nMelhorarA+=1
			nMudarMaoA+=1

		elif (categoriaMaoPC_ordenada == categoria_novaMaoA):

			#Usar função MaoEquivalente para saber qual é a mão equivalente à mão do PC e qual a mão eq. à possível nova mão.
			eqMaoPC= MaoEquivalente(maoPC_ordenada)[1]
			eqNovaMaoA= MaoEquivalente(novaMaoA)[1]

			#Usar função compara_varios para decidir qual a melhor mão: eqMaoPC ou eqNovaMaoA 

			melhorMao= compara_varios([eqMaoPC,eqNovaMaoA]) # melhorMao= melhor mão de entre os 2 inputs, ou 0 para informar que estão empatados.

			if (melhorMao == eqNovaMaoA):

				nMelhorarA+=1

		# Investigar probabilidades para o caso de descartarmos a carta B:

		categoria_novaMaoB= MaoEquivalente(novaMaoB)[0]

		if (categoriaMaoPC_ordenada < categoria_novaMaoB):
		
			nMelhorarB+=1
			nMudarMaoB+=1

		elif (categoriaMaoPC_ordenada == categoria_novaMaoB):

			#Usar função MaoEquivalente para saber qual é a mão equivalente à mão do PC e qual a mão eq. à possível nova mão.

			eqMaoPC= MaoEquivalente(maoPC_ordenada)[1]
			eqNovaMaoB= MaoEquivalente(novaMaoB)[1]

			#Usar função compara_varios para decidir qual a melhor mão: eqMaoPC ou eqNovaMaoB 

			melhorMao= compara_varios([eqMaoPC,eqNovaMaoB]) # melhorMao= melhor mão de entre os 2 inputs, ou 0 para informar que estão empatados.

			if (melhorMao == eqNovaMaoB):

				nMelhorarB+=1

		# Investigar probabilidades para o caso de descartarmos a carta C:

		categoria_novaMaoC= MaoEquivalente(novaMaoC)[0]

		if (categoriaMaoPC_ordenada < categoria_novaMaoC):
			
			nMelhorarC+=1
			nMudarMaoC+=1

		elif (categoriaMaoPC_ordenada == categoria_novaMaoC):

			#Usar função MaoEquivalente para saber qual é a mão equivalente à mão do PC e qual a mão eq. à possível nova mão.

			eqMaoPC= MaoEquivalente(maoPC_ordenada)[1]
			eqNovaMaoC= MaoEquivalente(novaMaoC)[1]

			#Usar função compara_varios para decidir qual a melhor mão: eqMaoPC ou eqNovaMaoC 

			melhorMao= compara_varios([eqMaoPC,eqNovaMaoC]) # melhorMao= melhor mão de entre os 2 inputs, ou 0 para informar que estão empatados.

			if (melhorMao == eqNovaMaoC):

				nMelhorarC+=1

		# Investigar probabilidades para o caso de descartarmos a carta D:

		categoria_novaMaoD= MaoEquivalente(novaMaoD)[0]

		if (categoriaMaoPC_ordenada < categoria_novaMaoD):
			
			nMelhorarD+=1
			nMudarMaoD+=1

		elif (categoriaMaoPC_ordenada == categoria_novaMaoD):

			#Usar função MaoEquivalente para saber qual é a mão equivalente à mão do PC e qual a mão eq. à possível nova mão.

			eqMaoPC= MaoEquivalente(maoPC_ordenada)[1]
			eqNovaMaoD= MaoEquivalente(novaMaoD)[1]

			#Usar função compara_varios para decidir qual a melhor mão: eqMaoPC ou eqNovaMaoC 

			melhorMao= compara_varios([eqMaoPC,eqNovaMaoD]) # melhorMao= melhor mão de entre os 2 inputs, ou 0 para informar que estão empatados.

			if (melhorMao == eqNovaMaoD):

				nMelhorarD+=1
		


	#Calculamos probabilidades de melhorar o tipo da mão/ melhorar a mão em geral para cada carta que se pode tirar: 
	#pA_MudarMao, pA_Melhorar, pB_MudarMao, pB_Melhorar, pC_MudarMao, pC_Melhorar, pD_MudarMao, pD_Melhorar

	#pN_MudarMao= prob de melhorar o tipo da mão descartando a carta N: nMudarMaoN/ length(cartasPossiveis)

	#pN_Melhorar= prob de melhorar a mão, no geral, descartando a carta N: nMelhorarN/ length(cartasPossiveis)

	pA_MudarMao= nMudarMaoA/len(cartasPossiveis)

	pA_Melhorar= nMelhorarA/ len(cartasPossiveis)


	pB_MudarMao= nMudarMaoB/len(cartasPossiveis)

	pB_Melhorar= nMelhorarB/ len(cartasPossiveis)


	pC_MudarMao= nMudarMaoC/len(cartasPossiveis)

	pC_Melhorar= nMelhorarC/ len(cartasPossiveis)


	pD_MudarMao= nMudarMaoD/len(cartasPossiveis)

	pD_Melhorar= nMelhorarD/ len(cartasPossiveis)

	#Decidir a partir das probabilidades se devemos deitar alguma fora, e qual.

	boasCandidatas=[] #lista dos índices das cartas que são boas candidatas para serem descartadas

	probMinima_Descartar = 0.5

	if (pA_Melhorar > probMinima_Descartar): #carta A é boa candidata para ser descartada

		boasCandidatas.append(0)

	if (pB_Melhorar > probMinima_Descartar): #carta B é boa candidata para ser descartada

		boasCandidatas.append(1)

	if (pC_Melhorar > probMinima_Descartar): #carta C é boa candidata para ser descartada

		boasCandidatas.append(2)

	if (pD_Melhorar > probMinima_Descartar): #carta D é boa candidata para ser descartada

		boasCandidatas.append(3)

	#Retornar lista com índice da carta da lista maoPC_ordenada que é para trocar, e com respetiva prob. de mudar o tipo de mão. Ex: se for para deitar fora a carta A, retornamos [0].

	if (boasCandidatas!=[]):

		#Determinar qual, de entre as boas candidatas, tem mais prob de melhorar o tipo da mão.

		probs_MudarMao= [pA_MudarMao,pB_MudarMao,pC_MudarMao,pD_MudarMao]

		max=0
		maisProvavelMudarMao= -1 #índice da carta boa candidata que mais provavelmente muda o tipo de mão.
		
		for i in boasCandidatas:

			if (probs_MudarMao[i]>=max):

				max= probs_MudarMao[i]
				maisProvavelMudarMao= i

		return [[maisProvavelMudarMao], probs_MudarMao[maisProvavelMudarMao]]

	else:

		return[]





def decideDraw_2cartas(maoPC_ordenada, cartasPossiveis):
	
	#Descartando 2, quais devemos descartar (se é que devemos descartar)?

	#maoPC_ordenada=[A,B,C,D] # mão atual do PC por ordem crescente de rank

	#combinacoesPossiveisDescartar= Combinaçoes dos indices das cartas que podemos descartar: [[0,1],[0,2],...]
	#novaMaoCombinacao1 = [aleatória, aleatoria, C, D], com Combinacao1= [0,1]

	#cartasPossiveis = [cartas que podem substituir a carta que estamos a descartar]
	#paresPossiveis= [pares de cartas que podem substituir o par descartado]

	#nMudarMao[n]= numero de pares de cartas possiveis que mudam o tipo de mão para melhor, ao descartar a combinação n
	#nMelhorar[n]= numero de pares de cartas possiveis que mudam a mão para melhor, no geral (seja melhorando o tipo de mão, seja reduzindo a carta maior), ao descartar a combinação n
	
	indices=[0,1,2,3]

	combinacoesPossiveisDescartar= combinacoesSemRepeticoes(2, indices)
	paresPossiveis= list(itertools.combinations(iterable=cartasPossiveis,r=2))

	#Numero de combinaçoes possiveis de descartar:
	N= len(combinacoesPossiveisDescartar)
	
	nMudarMao= np.zeros(N)
	nMelhorar= np.zeros(N) 

	for par in paresPossiveis: #ex: par=['DO','RP']
		
		for n in range(N):
			
			combinacaoDescartar= combinacoesPossiveisDescartar[n] #ex: combinacaoDescartar= [0,1]
			
			cartasParaDescartar= [] #cartas que são para descartar
			
			for indice in combinacaoDescartar:
				cartasParaDescartar.append(maoPC_ordenada[indice])

			novaMao= maoPC_ordenada[:]
			for carta in cartasParaDescartar: #ex: cartasParaDescartar= ['4E','DC']
				novaMao.remove(carta)
			
			for cartaNova in par:
				novaMao.append(cartaNova)

			# Investigar probabilidades:

			categoriaMaoPC_ordenada= MaoEquivalente(maoPC_ordenada)[0]
			categoria_novaMao= MaoEquivalente(novaMao)[0]
		
		if (categoriaMaoPC_ordenada < categoria_novaMao):
			
			nMelhorar[n]+=1
			nMudarMao[n]+=1

		elif (categoriaMaoPC_ordenada == categoria_novaMao):

			#Usar função MaoEquivalente para saber qual é a mão equivalente à mão do PC e qual a mão eq. à possível nova mão.
			eqMaoPC= MaoEquivalente(maoPC_ordenada)[1]
			eqNovaMao= MaoEquivalente(novaMao)[1]

			#Usar função compara_varios para decidir qual a melhor mão: eqMaoPC ou eqNovaMao 

			melhorMao= compara_varios([eqMaoPC,eqNovaMao]) # melhorMao= melhor mão de entre os 2 inputs, ou 0 para informar que estão empatados.

			if (melhorMao == eqNovaMao):

				nMelhorar+=1

	#Calculamos probabilidades de melhorar o tipo da mão/ melhorar a mão em geral para cada combinação de cartas que se pode tirar: 

	p_Melhorar= np.zeros(N)
	p_MudarMao= np.zeros(N)

	#p_MudarMao[n]= prob de melhorar o tipo da mão descartando a combinacao n: nMudarMao[n]/ length(paresPossiveis)

	#p_Melhorar[n]= prob de melhorar a mão, no geral, descartando a combinacao n: nMelhorar[n]/ length(paresPossiveis)

	for n in range(N):

		p_MudarMao[n]= nMudarMao[n]/ len(paresPossiveis)
		p_Melhorar[n]= nMelhorar[n]/ len(paresPossiveis)

	#Decidir a partir das probabilidades se devemos deitar alguma combinacao fora, e qual.

	boasCandidatas=[] #lista dos n correspondentes as combinacoes de índices das cartas que são boas candidatas para serem descartadas

	probMinima_Descartar = 0.5

	for n in range(N):

		if (p_Melhorar[n] > probMinima_Descartar): 

			boasCandidatas.append(n)

	#Retornar lista com a combinacao de índices das cartas da lista maoPC_ordenada que é para trocar. Ex: se for para deitar fora as carta A e B, retornamos [0,1].

	if (boasCandidatas!=[]):

		#Determinar qual combinacao, de entre as boas candidatas, tem mais prob de melhorar o tipo da mão.

		max=0
		maisProvavelMudarMao= -1 #n correspondente à combinacao de índices boa candidata que mais provavelmente muda o tipo de mão.
		
		for i in boasCandidatas:

			if (p_MudarMao[i]>=max):

				max= p_MudarMao[i]
				maisProvavelMudarMao= i

		return [combinacoesPossiveisDescartar[maisProvavelMudarMao],p_MudarMao[maisProvavelMudarMao]]

	else:

		return[]


############################################################################################################################
############################################################################################################################
############################################################################################################################

def decideDraw(maoPC_ordenada, cartasPossiveis):
	umaCarta= decideDraw_1carta(maoPC_ordenada, cartasPossiveis) #[[x],p_x]
	duasCartas= decideDraw_2cartas(maoPC_ordenada, cartasPossiveis) #[[x,y],p_x,y]
	lista= []
	lista.append(umaCarta) 
	lista.append(duasCartas)
	if (lista[0]==[] and lista[1]==[]):
		return []
	elif (lista[0]==[]):
		return lista[1][0]
	elif (lista[1]==[]):
		return lista[0][0]
	elif (lista[0][1]==lista[1][1]):
		return lista[0][0]
	elif (lista[0][1]>lista[1][1]):
		return lista[0][0]
	else:
		return lista[1][0]




################################################# Funcao drawPC #############################################################

#cartasPc,cartasPossiveis,baralho,cartasDescartadasPC
def drawPC(maoPC_ordenada, cartasPossiveis, baralhoAtual, cartasDescartadasPC):

	baralhoCompleto= ['2C','2O','2E','2P','3C','3O','3E','3P','4C','4O','4E','4P','5C','5O','5E','5P',
	'6C','6O','6E','6P','7C','7O','7E','7P','8C','8O','8E','8P','9C','9O','9E','9P','10C','10O','10E',
	'10P','DC','DO','DE','DP','VC','VO','VE','VP','RC','RO','RE','RP','AC','AO','AE','AP']

	l= decideDraw(maoPC_ordenada, cartasPossiveis) #lista com posições das cartas da lista maoPC_ordenada que são para trocar
	
	if (l!=[]):

		cartasParaDescartar= [] #cartas que são para descartar

		for i in l: #ex:l= [0,2]
			cartasParaDescartar.append(maoPC_ordenada[i])

		for carta in cartasParaDescartar: #ex: cartasParaDescartar= ['4E','DC']
			maoPC_ordenada.remove(carta)
			cartasDescartadasPC.append(carta)

		numCartas= len(l)
		[novasCartas,baralhoAtual]= retira_n_cartas(baralhoAtual, numCartas)

		for novaCarta in novasCartas:
			maoPC_ordenada.append(novaCarta) 

		#no final deste for, a maoPC_ordenada pode não estar ordenada.

		cartasNaoPossiveis= maoPC_ordenada + cartasDescartadasPC 
		cartasPossiveis= removerCartas(baralhoCompleto, cartasNaoPossiveis)

	else:
		numCartas=0 

	print("\nNúmero de cartas trocadas pelo seu oponente: "+str(numCartas)+"\n") #num de cartas trocadas pelo PC

	return [maoPC_ordenada, cartasPossiveis, baralhoAtual, cartasDescartadasPC]



################################################## FUNÇÃO DRAW_jogador ########################################################## (foi criada uma que só permite tirar uma carta, mas está no colab do badugi)

def draw_jogador(baralho,mao,CartasRetiradas):
  a = int(input("Quantas cartas quer trocar?")) ## deve ser só para trocar uma
  if(a>0 and a<=len(mao)):
    #s = input("Quais cartas quer trocar? ")
    #s = input("Quais cartas quer trocar?\n Exemplo: x ou x,y ou x,y,z ou x,y,z,r(em que x,y,z,r pertencem respetivamente ao conjunto [0,1,2,3])")
    s=input("Selecione a posição/ões da/s carta/s a trocar separadas por vírgulas: ")
    lista = re.split(r',',s)
    true_list = list(map(int,lista))
    if(len(true_list)!=a):
      print("Erro:Coloque valores válidos")
      x=draw_jogador(baralho,mao,CartasRetiradas)
      return[x[0],x[1],x[2]]
    conj = retira_n_cartas(baralho,a) #conjunto de cartas que vão trocar com as da mao
    CartasRetiradas.append(a)


    retira = []
    for i in range(len(true_list)):
      b = true_list[i]
      retira.append(mao[b])
      mao[b] = conj[0][0]
      conj[0].remove(conj[0][0])
    #final = removerCartas(conj[1],retira)

    return [mao,conj[1],CartasRetiradas]
  elif(a>len(mao) or a<0):
    print("Número de cartas a trocar inválidas !")
    x=draw_jogador(baralho,mao,CartasRetiradas)
    return[x[0],x[1],x[2]]
  else:
    CartasRetiradas.append(0)
    return[mao,baralho,CartasRetiradas]





################ Retira n cartas ##################

#def retira_n_cartas (baralho,n):
#  baralho_aux=baralho
#  print(baralho_aux)
#  if(n>len(baralho_aux)):
#    return [[],baralho_aux]
#    print("inválido, não se pode escolher mais cartas do que as que existem")
#  else:
#    if (n==0):
#      return [[],baralho_aux]
#    else:
#      
#      t=len(baralho_aux)
#      mao=[]
#      c=1
#      while (c):
#        dist=tfd.Categorical(probs=[1/t]*t)
#        samples=dist.sample(n)
#        samples_array=samples.numpy().tolist()
#        aux=sorted(set(samples_array))
#        if (len(aux)==n):
#          c=0
#        #print('debugg:',samples_array)
#      for i in range(n):
#        mao.append(baralho_aux[samples_array[i]])
#      #print('debugcards',mao)
#      for j in range(n):
#        #print(mao[j])
#        baralho.remove(mao[j])
#      
#      return[mao,baralho]


########################################################### função tira  ##################################################

def tira(a,b):
   
    for i in range(len(b)):
        a.remove(b[i])
    return a


# mao_Jogador,mao_PC,cartasDescartadasPC,baralho_atualizado,Cartaspossiveis,CartasRetiradas
############################################################ Função draw ####################################################

#mao_Jogador,mao_PC,cartasDescartadasPC,baralho_atualizado,Cartaspossiveis,CartasRetiradas
def draw(cartasJogador,cartasPc,cartasDescartadasPC,baralho,cartasPossiveis,CartasRetiradas):

  print("############### Trocar cartas ###############")

  a = draw_jogador(baralho,cartasJogador,CartasRetiradas)
  #output: mao_novaJogador,baralho_atualizado,CartasRetiradas


  # acho que a variavel "baralho" tem de ser a o output a[1]
  b = drawPC(cartasPc,cartasPossiveis,baralho,cartasDescartadasPC) #[maoPC,cartasPossiveis,baralho,cartasdescartadas]
  #[maoPC_ordenada, cartasPossiveis, baralhoAtual, cartasDescartadasPC]

  return [a[0],a[1],a[2],b[0],b[3],b[2],b[1]]
  # output: mao_novaJogador,baralho_atualizadoJogador,CartasRetiradasJogador,mao_novaPC,cartasFora,Baralho_atualPC,Baralho_semMaoPC_e_cartasDescartadasPC

################################################## Funcao showDown #########################################################

def showDown(maoJogador, maoPC, dinheiroPote, dinheiroJogador, dinheiroPC):

	print("------------------- Showdown! -------------------\n")

	print("Sua mão: "+str(maoJogador)+"\n")
	print("Mão do seu oponente: "+str(maoPC)+"\n")

	categoriaMaoPC= MaoEquivalente(maoPC)[0]
	categoriaMaoJogador= MaoEquivalente(maoJogador)[0]

	# o jogador venceu o jogo
	if (categoriaMaoPC < categoriaMaoJogador): 
		dinheiroJogador += dinheiroPote
		print("Você venceu o jogo! Bom jogo, parabéns! \nO dinheiro no pote é "+str(dinheiroPote)+".\nO seu saldo é "+str(dinheiroJogador)+" euros.\nO saldo do seu oponente é "+str(dinheiroPC)+" euros.")
		#return "Jogador"

	#Usar função MaoEquivalente para saber qual é a mão equivalente à mão do PC e qual a mão eq. à do Jogador.
	#Usar função comparaMaos para decidir qual a melhor mão: eqMaoPC ou eqMaoJogador
	#as mãos do jogador e do PC têm o mesmo tipo

	elif (categoriaMaoPC == categoriaMaoJogador): 
		
		eqMaoPC= MaoEquivalente(maoPC)[1]
		eqMaoJogador= MaoEquivalente(maoJogador)[1]
		
		melhorMao= compara_varios([eqMaoPC,eqMaoJogador]) # melhorMao= melhor mão de entre os 2 inputs, ou 0 para informar que estão empatados.

		if (melhorMao == eqMaoPC):
			dinheiroPC += dinheiroPote
			print("Você perdeu o jogo! Bom jogo, tente novamente! \nO seu saldo é "+str(dinheiroJogador)+" euros.\nO saldo do seu oponente é "+str(dinheiroPC)+" euros.")
			#return "PC"
		
		elif (melhorMao == eqMaoJogador):
			dinheiroJogador += dinheiroPote
			print("Você venceu o jogo! Bom jogo, parabéns! \nO dinheiro no pote é "+str(dinheiroPote)+".\nO seu saldo é "+str(dinheiroJogador)+" euros.\nO saldo do seu oponente é "+str(dinheiroPC)+" euros.")
			#return "Jogador"

		elif (melhorMao == 0):
			dinheiroJogador += dinheiroPote/2.0
			dinheiroPC += dinheiroPote/2.0
			print("Você e o seu oponente empataram! Bom jogo, tente novamente! \nO dinheiro no pote é "+str(dinheiroPote)+".\nO seu saldo é "+str(dinheiroJogador)+" euros.\nO saldo do seu oponente é "+str(dinheiroPC)+" euros.")
			#return "Empate"

	# o PC venceu o jogo
	else:
		dinheiroPC += dinheiroPote
		print("Você perdeu o jogo! Bom jogo, tente novamente! \nO seu saldo é "+str(dinheiroJogador)+" euros.\nO saldo do seu oponente é "+str(dinheiroPC)+" euros.")
		#return "PC"
	
	return[dinheiroPC,dinheiroJogador]






















In [7]:
#########################################################################################################################
################################################################### ApostaPC #########################################
##########################################################################################################################

def ApostaPC(apostaPC,apostaJogador,dinheiroPC,dinheiroJogador,ronda,maoPC,pesos_Valor,historicoJogador,CartasRetiradas):

  #0->check
  #1->call
  #2->raise

  ## primeiro vou retirar os parasitas da mão do PC e ver a que categoria pertence
  # AC 2C 2O RP--> 
  categoria_maoSemParasitas=MaoEquivalente(maoPC)
  categoria=categoria_maoSemParasitas[0]
  mao_sem_parasitas=categoria_maoSemParasitas[1]
  mao_sem_parasitas.sort(key=retorna_dic_1)

  #print(categoria)

  #print(mao_sem_parasitas)

  #resto do dinheiro que o PC tem
  resto = dinheiroPC - apostaPC

  #O que falta pagar para igualar a aposta do jogador
  FaltaPagar = apostaJogador - apostaPC

  if(FaltaPagar>=resto):
    # se o PC quer continuar tem de dar all_in
    #Temos de ver se temos uma boa mao. Caso tenhemos uma boa mao entao da-mos all in. Se temos pouca probabilidade de ganhar da-mos fold

    #print("######## FaltaPagar>=resto ############")

    if(categoria==4):
      # a probabilidade de dar fold vai variar em função da força da minha mao
     # print("######## Badugi ############")
      valores=retira_valores(mao_sem_parasitas)

      #print(valores)
      
      # A 3 5 R
      # 5 6 7 8
      valor_deck=pesos_valor[valores[0]]+10*pesos_valor[valores[1]]+100*pesos_valor[valores[2]]+1000*pesos_valor[valores[3]]

      #print("valor_deck:",valor_deck)


      if(ronda==1):
        #Se tivermos na ronda 1 os parametros da prob de dar fold vao ter um certo peso
       # print("######## Ronda 1 ############")
        if(len(historicoJogador)==0):
          #se o jogador aind não apostou nada
          prob_fold=((apostaJogador/dinheiroJogador)*0.4)*(1-(valor_deck-1234)/9999)
         # print("prob_fold",prob_fold)

        else:
          #Caso já exista apostas na mesma ronda
          prob_fold=((apostaJogador/dinheiroJogador)*0.4+((sum(historicoJogador))/(2*len(historicoJogador)))*0.1)*(1-(valor_deck-1234)/9999)
         # print("prob_fold",prob_fold)
        

      elif(ronda==2):
       # print("######## Ronda 2 ############")
                                                                                                                     ### [4,3]
        #Caso já exista apostas na mesma ronda
        prob_fold=((apostaJogador/dinheiroJogador)*0.4 + ((sum(historicoJogador))/(2*len(historicoJogador)))*0.2 + ((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.1)*(1-(valor_deck-1234)/9999)
        #print("prob_fold",prob_fold)
        
        
      elif(ronda==3):
        #print("######## Ronda 3 ############")

        #Caso já exista apostas na mesma ronda
        prob_fold=((apostaJogador/dinheiroJogador)*0.4 + ((sum(historicoJogador))/(2*len(historicoJogador)))*0.2 + ((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.2)*(1-(valor_deck-1234)/9999)
        #print("prob_fold",prob_fold)
        

      elif(ronda==4):
        #print("######## Ronda 4 ############")

        #Caso já exista apostas na mesma ronda
        prob_fold=((apostaJogador/dinheiroJogador)*0.4 + ((sum(historicoJogador))/(2*len(historicoJogador)))*0.2 + ((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.2)*(1-(valor_deck-1234)/9999)
       # print("prob_fold",prob_fold)
        
      #basicamente é um flip
      dist_fold=tfd.Binomial(1.,probs=prob_fold)
      fold_samples = dist_fold.sample(1)
      fold_PC_list=fold_samples.numpy().tolist()
      fold_PC=fold_PC_list[0]
     # print("fold_PC:",fold_PC)

      if(fold_PC):
        #o PC desitiu
        print("O PC deu Fold!")
        apostaPC = apostaPC
        all_inPC = 0
        foldPC = 1

      else:
        print("O PC deu All_in")
        #O PC quer ir a jogo
        apostaPC = dinheiroPC
        all_inPC = 1
        foldPC = 0


####################### Categoria==3 - mao de 3 cartas ###########################


    if(categoria==3):
      # a probabilidade de dar fold vai variar em função da força da minha mao
      #print("######## mao de 3 cartas ############")
      valores=retira_valores(mao_sem_parasitas)
      valor_deck=pesos_valor[valores[0]]+10*pesos_valor[valores[1]]+100*pesos_valor[valores[2]]
      #print("valor_deck:",valor_deck)


      if(ronda==1):
        #Se tivermos na ronda 1 os parametros da prob de dar fold vao ter um certo peso
        #print("######## Ronda 1 ############") 

        if(len(historicoJogador)==0):
          #se o jogador aind não apostou nada
          prob_fold=((apostaJogador/dinheiroJogador)*0.6)+(1-(valor_deck-123)/1110)*0.1
          #print("prob_fold",prob_fold)

        else:
          #Caso já exista apostas na mesma ronda
          prob_fold=(apostaJogador/dinheiroJogador)*0.6+((sum(historicoJogador))/(2*len(historicoJogador)))*0.1 + (1-(valor_deck-123)/1110)*0.1
          #print("prob_fold",prob_fold)
        

      elif(ronda==2):
        #print("######## Ronda 2 ############")

        #P_fold = (1- (pontNossaMao- 123)/1110 )*0.1+(valor_aposta_dele/dinheiro_dele)*0.6+(apostas_anteriores/apostas_maximas)*0.1+(1-cartasTrocadasPeloJogador/n_rondas)*0.1

        #Caso já exista apostas na mesma ronda
        prob_fold=(apostaJogador/dinheiroJogador)*0.6+((sum(historicoJogador))/(2*len(historicoJogador)))*0.1+((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.1+(1-(valor_deck-123)/1110)*0.1
        #print("prob_fold",prob_fold)
        
        
      elif(ronda==3):
        #print("######## Ronda 3 ############")
        #Caso já exista apostas na mesma ronda
        prob_fold=(apostaJogador/dinheiroJogador)*0.6+((sum(historicoJogador))/(2*len(historicoJogador)))*0.2+((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.1+(1-(valor_deck-123)/1110)*0.1
        #print("prob_fold",prob_fold)
        

      elif(ronda==4):
        #print("######## Ronda 4 ############")

        #Caso já exista apostas na mesma ronda
        prob_fold=(apostaJogador/dinheiroJogador)*0.6+((sum(historicoJogador))/(2*len(historicoJogador)))*0.2+((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.1+(1-(valor_deck-123)/1110)*0.1
        #print("prob_fold",prob_fold)
        
      #basicamente é um flip
      dist_fold=tfd.Binomial(1.,probs=prob_fold)
      fold_samples = dist_fold.sample(1)
      fold_PC_list=fold_samples.numpy().tolist()
      fold_PC=fold_PC_list[0]
      #print("fold_PC:",fold_PC)

      if(fold_PC):
        print("O PC deu Fold!")
        #o PC desitiu
        apostaPC = apostaPC
        all_inPC = 0
        foldPC = 1

      else:
        print("O PC deu All_in!")
        #O PC quer ir a jogo
        apostaPC = dinheiroPC
        all_inPC = 1
        foldPC = 0


##################### Mao de 2 cartas ################ 

    if(categoria==2):
      # a probabilidade de dar fold vai variar em função da força da minha mao
      #print("######## mao de 2 cartas ############")
      #valores=retira_valores(mao_sem_parasitas)
      #valor_deck=pesos_valor[valores[0]]+10*pesos_valor[valores[1]]
      #print("valor_deck:",valor_deck)


      if(ronda==1):
        #Se tivermos na ronda 1 os parametros da prob de dar fold vao ter um certo peso
       # print("######## Ronda 1 ############")  

        if(len(historicoJogador)==0):
          #se o jogador aind não apostou nada
          #prob_fold=((apostaJogador/dinheiroJogador)*0.7)+(1-(valor_deck-12)/121)*0.1
          prob_fold=0.9
         # print("prob_fold",prob_fold)

        else:
          #Caso já exista apostas na mesma ronda
          #prob_fold=(apostaJogador/dinheiroJogador)*0.7+((sum(historicoJogador))/(2*len(historicoJogador)))*0.1 + (1-(valor_deck-12)/121)*0.1
          prob_fold=0.9
         # print("prob_fold",prob_fold)
        

      elif(ronda==2):
       # print("######## Ronda 2 ############")

        #Caso já exista apostas na mesma ronda
        #prob_fold=(apostaJogador/dinheiroJogador)*0.7+((sum(historicoJogador))/(2*len(historicoJogador)))*0.1+(1-sum(CartasRetiradas)/len(CartasRetiradas))*0.1+(1-(valor_deck-12)/121)*0.1
        prob_fold=0.95
       # print("prob_fold",prob_fold)
        
        
      elif(ronda==3):
       # print("######## Ronda 3 ############")

        #Caso já exista apostas na mesma ronda
        #prob_fold=(apostaJogador/dinheiroJogador)*0.6+((sum(historicoJogador))/(2*len(historicoJogador)))*0.2+(1-sum(CartasRetiradas)/len(CartasRetiradas))*0.1+(1-(valor_deck-12)/121)*0.1
        prob_fold=1
        #print("prob_fold",prob_fold)
        

      elif(ronda==4):
        #print("######## Ronda 4 ############")

        #Caso já exista apostas na mesma ronda
        #prob_fold=(apostaJogador/dinheiroJogador)*0.6+((sum(historicoJogador))/(2*len(historicoJogador)))*0.2+(1-sum(CartasRetiradas)/len(CartasRetiradas))*0.1+(1-(valor_deck-12)/121)*0.1
        prob_fold=1
        #print("prob_fold",prob_fold)
        
      #basicamente é um flip
      dist_fold=tfd.Binomial(1.,probs=prob_fold)
      fold_samples = dist_fold.sample(1)
      fold_PC_list=fold_samples.numpy().tolist()
      fold_PC=fold_PC_list[0]
     # print("fold_PC:",fold_PC)

      if(fold_PC):
        #o PC desitiu
        print("O PC deu Fold!")
        apostaPC = apostaPC
        all_inPC = 0
        foldPC = 1

      else:
        #O PC quer ir a jogo
        print("O PC deu All_in!")
        apostaPC = dinheiroPC
        all_inPC = 1
        foldPC = 0


##################### Mao de 1 carta ################ 

    if(categoria==1):
      # a probabilidade de dar fold vai variar em função da força da minha mao
      #print("######## mao de 1 carta ############")

      #A prob de ganhar com uma mao de 1 carta é praticamente zero => queremos dar fold

      if (ronda==1):
        prob_fold=0.95
      else:
        prob_fold=1

      #print("prob_fold:",prob_fold)

      dist_fold=tfd.Binomial(1.,probs=prob_fold)
      fold_samples = dist_fold.sample(1)
      fold_PC_list=fold_samples.numpy().tolist()
      fold_PC=fold_PC_list[0]
      #print("fold_PC:",fold_PC)

      if(fold_PC):
        #o PC desitiu
        print("O PC deu Fold!")
        apostaPC = apostaPC
        all_inPC = 0
        foldPC = 1

      else:
        print("O PC deu All_in!")
        #O PC quer ir a jogo
        apostaPC = dinheiroPC
        all_inPC = 1
        foldPC = 0

############### FaltaPagar < resto ##########

  else: 
    # FaltaPagar<resto => o pc tem dinheiro para pagar o que falta para igualar a aposta do jogador
    
    ########### FaltaPagar ==0 #############
    if(FaltaPagar==0):
      # podemos dar check, ou dar raise ou all in

      #print("######## FaltaPagar = 0 ############")


      ##############################temos um badugi###################
      if(categoria==4):
        
        #print("######## Badugi ############")
        valores=retira_valores(mao_sem_parasitas)
        valor_deck=pesos_valor[valores[0]]+10*pesos_valor[valores[1]]+100*pesos_valor[valores[2]]+1000*pesos_valor[valores[3]]

        #print("Valor deck: ",valor_deck)

        if(ronda==1):
          #print("ronda 1")

          if(len(historicoJogador)==0):
            p_allinPC = ((valor_deck- 1234)/9999 )*0.1 + 0.1
            #print("p_allinPC:",p_allinPC)
          else:
            p_allinPC = ((valor_deck- 1234)/9999 )*0.1 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.1
            #print("p_allinPC:",p_allinPC)

          dist_allin=tfd.Binomial(1.,probs=p_allinPC)
          allin_samples = dist_allin.sample(1)
          allin_PC_list=allin_samples.numpy().tolist()
          all_inPC=allin_PC_list[0]
          #print("all_inPC:",all_inPC)

          if(all_inPC==1):
            print("O PC deu All_in!")
            apostaPC = dinheiroPC
            all_inPC = 1
            foldPC = 0
          else:
            # agora vamos ver se vamos dar raise:
           
            p_raise=((valor_deck- 1234)/9999 )*0.4 +0.2
            #print("p_raise:",p_raise)

            dist_raise=tfd.Binomial(1.,probs=p_raise)
            raise_samples = dist_raise.sample(1)
            raisePC_list=raise_samples.numpy().tolist()
            raisePC=raisePC_list[0]
            #print("raisePC:",raisePC)

            if(raisePC==1):
              print("O PC deu Raise!")
              apostaPC = apostaPC + int(0.05*resto)
              all_inPC = 0
              foldPC = 0
            else:
              print("O PC deu Check!")
              # então deu check
              apostaPC = apostaPC
              all_inPC = 0
              foldPC = 0


        if(ronda==2):
          #print("ronda 2")
          #P all_in: ((pontNossaMao- 1234)/9999 )*0.2 +(1-apostas_anteriores/apostas_maximas)*0.1+ cartasTrocadasPeloJogador/n_rondas*0.1+(1- dinheiroDiponivel/dinhero que tinha no inicio)*0.2


          if(len(historicoJogador)==0):
            p_allinPC = ((valor_deck- 1234)/9999 )*0.1 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.1
            #print("p_allinPC:",p_allinPC)
          else:
            p_allinPC = ((valor_deck- 1234)/9999 )*0.1 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.05 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.05
            #print("p_allinPC:",p_allinPC)

          dist_allin=tfd.Binomial(1.,probs=p_allinPC)
          allin_samples = dist_allin.sample(1)
          allin_PC_list=allin_samples.numpy().tolist()
          all_inPC=allin_PC_list[0]
          #print("all_inPC:",all_inPC)

          if(all_inPC==1):
            print("O PC deu All_in!")
            apostaPC = dinheiroPC
            all_inPC = 1
            foldPC = 0
          else:
            # agora vamos ver se vamos dar raise:
            
            p_raise=((valor_deck- 1234)/9999 )*0.4 +0.2
            #print("p_raise:",p_raise)

            dist_raise=tfd.Binomial(1.,probs=p_raise)
            raise_samples = dist_raise.sample(1)
            raisePC_list=raise_samples.numpy().tolist()
            raisePC=raisePC_list[0]
            #print("raisePC:",raisePC)

            if(raisePC==1):
              print("O PC deu Raise!")
              apostaPC = apostaPC + int(0.05*resto)
              all_inPC = 0
              foldPC = 0
            else:
              print("O PC deu Check!")
              # então deu check
              apostaPC = apostaPC
              all_inPC = 0
              foldPC = 0



          

        if(ronda==3):
          #print("ronda 3")
        
          if(len(historicoJogador)==0):
            p_allinPC = ((valor_deck- 1234)/9999 )*0.05 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.15
            #print("p_allinPC:",p_allinPC)
          else:
            p_allinPC = ((valor_deck- 1234)/9999 )*0.05 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.1 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.05
           # print("p_allinPC:",p_allinPC)

          dist_allin=tfd.Binomial(1.,probs=p_allinPC)
          allin_samples = dist_allin.sample(1)
          allin_PC_list=allin_samples.numpy().tolist()
          all_inPC=allin_PC_list[0]
          #print("all_inPC:",all_inPC)

          if(all_inPC==1):
            print("O PC deu All_in!")
            apostaPC = dinheiroPC
            all_inPC = 1
            foldPC = 0
          else:
            # agora vamos ver se vamos dar raise:
           
            p_raise=((valor_deck- 1234)/9999 )*0.4 +0.2
            #print("p_raise:",p_raise)

            dist_raise=tfd.Binomial(1.,probs=p_raise)
            raise_samples = dist_raise.sample(1)
            raisePC_list=raise_samples.numpy().tolist()
            raisePC=raisePC_list[0]
            #print("raisePC:",raisePC)

            if(raisePC==1):
              print("O PC deu Raise!")
              apostaPC = apostaPC + int(0.05*resto)
              all_inPC = 0
              foldPC = 0
            else:
              print("O PC deu Check!")
              # então deu check
              apostaPC = apostaPC
              all_inPC = 0
              foldPC = 0




        if(ronda==4):
          #print("ronda 4")
       
          if(len(historicoJogador)==0):
            p_allinPC = ((valor_deck- 1234)/9999 )*0.05 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.15
            #print("p_allinPC:",p_allinPC)
          else:
            p_allinPC = ((valor_deck- 1234)/9999 )*0.05 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.1 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.05
            #print("p_allinPC:",p_allinPC)

          dist_allin=tfd.Binomial(1.,probs=p_allinPC)
          allin_samples = dist_allin.sample(1)
          allin_PC_list=allin_samples.numpy().tolist()
          all_inPC=allin_PC_list[0]
          #print("all_inPC:",all_inPC)

          if(all_inPC==1):
            print("O PC deu All_in!")
            apostaPC = dinheiroPC
            all_inPC = 1
            foldPC = 0
          else:
            # agora vamos ver se vamos dar raise:
            # no máximo vai até 60%

            p_raise=((valor_deck- 1234)/9999 )*0.4 +0.2
            #print("p_raise:",p_raise)

            dist_raise=tfd.Binomial(1.,probs=p_raise)
            raise_samples = dist_raise.sample(1)
            raisePC_list=raise_samples.numpy().tolist()
            raisePC=raisePC_list[0]
           # print("raisePC:",raisePC)

            if(raisePC==1):
              print("O PC deu Raise!")
              apostaPC = apostaPC + int(0.05*resto)
              all_inPC = 0
              foldPC = 0
            else:
              print("O PC deu Check!")
              # então deu check
              apostaPC = apostaPC
              all_inPC = 0
              foldPC = 0







      ################## temos um triplo ###########################
      if(categoria==3):
        
       # print("######## triplo ############")
        valores=retira_valores(mao_sem_parasitas)
        valor_deck=pesos_valor[valores[0]]+10*pesos_valor[valores[1]]+100*pesos_valor[valores[2]]

        #print("Valor deck: ",valor_deck)

        if(ronda==1):
         # print("ronda 1")
         
          # no máximo vai até 10%
          if(len(historicoJogador)==0):
            p_allinPC = ((valor_deck- 123)/1110 )*0.05 + 0.05
            #print("p_allinPC:",p_allinPC)
          else:
            p_allinPC = ((valor_deck- 123)/1110 )*0.05 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.05
            #print("p_allinPC:",p_allinPC)

          dist_allin=tfd.Binomial(1.,probs=p_allinPC)
          allin_samples = dist_allin.sample(1)
          allin_PC_list=allin_samples.numpy().tolist()
          all_inPC=allin_PC_list[0]
          #print("all_inPC:",all_inPC)

          if(all_inPC==1):
            print("O PC deu All_in!")
            apostaPC = dinheiroPC
            all_inPC = 1
            foldPC = 0
          else:
            # agora vamos ver se vamos dar raise:
            
           # no máximo vai até 30% dependendo do triplo

            p_raise=((valor_deck- 123)/1110 )*0.2 +0.1
            #print("p_raise:",p_raise)

            dist_raise=tfd.Binomial(1.,probs=p_raise)
            raise_samples = dist_raise.sample(1)
            raisePC_list=raise_samples.numpy().tolist()
            raisePC=raisePC_list[0]
            #print("raisePC:",raisePC)


            # se der raise, aumento 5% do valor que ele tem para apostar
            if(raisePC==1):
              print("O PC deu Raise!")
              apostaPC = apostaPC + int(0.05*resto)
              all_inPC = 0
              foldPC = 0
            else:
              print("O PC deu Check!")
              # então deu check
              apostaPC = apostaPC
              all_inPC = 0
              foldPC = 0


        if(ronda==2):
         # print("ronda 2")
          
          if(len(historicoJogador)==0):
            p_allinPC = ((valor_deck- 123)/1110 )*0.05 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.05
            #print("p_allinPC:",p_allinPC)
          else:
            p_allinPC = ((valor_deck- 123)/1110 )*0.05 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.025 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.025
           # print("p_allinPC:",p_allinPC)

          dist_allin=tfd.Binomial(1.,probs=p_allinPC)
          allin_samples = dist_allin.sample(1)
          allin_PC_list=allin_samples.numpy().tolist()
          all_inPC=allin_PC_list[0]
          #print("all_inPC:",all_inPC)

          if(all_inPC==1):
            print("O PC deu All_in!")
            apostaPC = dinheiroPC
            all_inPC = 1
            foldPC = 0
          else:
            # agora vamos ver se vamos dar raise:


            p_raise=((valor_deck- 123)/1110 )*0.2 +0.1
            #print("p_raise:",p_raise)

            dist_raise=tfd.Binomial(1.,probs=p_raise)
            raise_samples = dist_raise.sample(1)
            raisePC_list=raise_samples.numpy().tolist()
            raisePC=raisePC_list[0]
            #print("raisePC:",raisePC)

            if(raisePC==1):
              print("O PC deu Raise!")
              apostaPC = apostaPC + int(0.05*resto)
              all_inPC = 0
              foldPC = 0
            else:
              print("O PC deu Check!")
              # então deu check
              apostaPC = apostaPC
              all_inPC = 0
              foldPC = 0



          

        if(ronda==3):
          #print("ronda 3")
         
          if(len(historicoJogador)==0):
            p_allinPC = ((valor_deck- 123)/1110 )*0.025 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.075
            #print("p_allinPC:",p_allinPC)
          else:
            p_allinPC = ((valor_deck- 123)/1110 )*0.025 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.05 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.025
            #print("p_allinPC:",p_allinPC)

          dist_allin=tfd.Binomial(1.,probs=p_allinPC)
          allin_samples = dist_allin.sample(1)
          allin_PC_list=allin_samples.numpy().tolist()
          all_inPC=allin_PC_list[0]
          #print("all_inPC:",all_inPC)

          if(all_inPC==1):
            print("O PC deu All_in!")
            apostaPC = dinheiroPC
            all_inPC = 1
            foldPC = 0
          else:
            # agora vamos ver se vamos dar raise:
            

            p_raise=((valor_deck- 123)/1110 )*0.2 +0.1
            #print("p_raise:",p_raise)

            dist_raise=tfd.Binomial(1.,probs=p_raise)
            raise_samples = dist_raise.sample(1)
            raisePC_list=raise_samples.numpy().tolist()
            raisePC=raisePC_list[0]
            #print("raisePC:",raisePC)

            if(raisePC==1):
              print("O PC deu Raise!")
              apostaPC = apostaPC + int(0.05*resto)
              all_inPC = 0
              foldPC = 0
            else:
              print("O PC deu Check!")
              # então deu check
              apostaPC = apostaPC
              all_inPC = 0
              foldPC = 0


        if(ronda==4):
          #print("ronda 4")
       
          if(len(historicoJogador)==0):
            p_allinPC = ((valor_deck- 123)/1110 )*0.025 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.075
            #print("p_allinPC:",p_allinPC)
          else:
            p_allinPC = ((valor_deck- 123)/1110 )*0.025 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.05 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.025
            #print("p_allinPC:",p_allinPC)

          dist_allin=tfd.Binomial(1.,probs=p_allinPC)
          allin_samples = dist_allin.sample(1)
          allin_PC_list=allin_samples.numpy().tolist()
          all_inPC=allin_PC_list[0]
          #print("all_inPC:",all_inPC)

          if(all_inPC==1):
            print("O PC deu All_in!")
            apostaPC = dinheiroPC
            all_inPC = 1
            foldPC = 0
          else:
            # agora vamos ver se vamos dar raise:
           

            p_raise=((valor_deck- 123)/1110 )*0.2 +0.1
            #print("p_raise:",p_raise)

            dist_raise=tfd.Binomial(1.,probs=p_raise)
            raise_samples = dist_raise.sample(1)
            raisePC_list=raise_samples.numpy().tolist()
            raisePC=raisePC_list[0]
            #print("raisePC:",raisePC)

            if(raisePC==1):
              print("O PC deu Raise!")

              apostaPC = apostaPC + int(0.05*resto)
              all_inPC = 0
              foldPC = 0
            else:
              print("O PC deu Check!")
              # então deu check
              apostaPC = apostaPC
              all_inPC = 0
              foldPC = 0

      ######################### Duplo #################################

      # aqui usaremos thresholds pois o modelo não se afinava como devia
      if(categoria==2):

        # estamos numa mao de 2 cartas
        #print("###### Duplo ######")

        # consideramos que é apenas estúpido dar all_in num duplo, por isso nem entra nas contas
        # ou seja ele dá check ou raise, com o check com probabilidade bué alta
        

        # prob de dar check de 95% para um duplo:
        dist_check=tfd.Binomial(1.,probs=0.95)
        check_samples = dist_check.sample(1)
        checkPC_list=check_samples.numpy().tolist()
        checkPC=checkPC_list[0]
        #print("checkPC:",checkPC)
        
        if(checkPC):
          print("O PC deu Check!")
          apostaPC = apostaPC
          all_inPC = 0
          foldPC = 0
        else:
          print("O PC deu Raise!")
          # dar raise para o bluff
          apostaPC = apostaPC + int(0.05*resto)
          all_inPC = 0
          foldPC = 0





        ######################### Solo #################################

       # aqui usaremos thresholds pois o modelo não se afinava como devia
      if(categoria==1):

        # estamos numa mao de 1 cartas
        #print("###### Solo ######")

        # consideramos que é apenas estúpido dar all_in num solo, por isso nem entra nas contas
        # ou seja ele dá check ou raise, com o check com probabilidade bué alta
        

        # prob de dar check de 99% para um duplo:
        dist_check=tfd.Binomial(1.,probs=0.99)
        check_samples = dist_check.sample(1)
        checkPC_list=check_samples.numpy().tolist()
        checkPC=checkPC_list[0]
        #print("checkPC:",checkPC)
        
        if(checkPC):
          print("O PC deu Check!")
          apostaPC = apostaPC
          all_inPC = 0
          foldPC = 0
        else:
          print("O PC deu Raise!")
          # dar raise para o bluff
          apostaPC = apostaPC + int(0.05*resto)
          all_inPC = 0
          foldPC = 0
    


    ######### FaltaPagar>0 e temos dinheiro para pagar o falta pagar##################
    else:
      #print("########### faltaPagar > 0 ###########")

      # 1---fold
      # 2--- all_in
      # 3-- raise
      # 4-- call
      

      ########### badugi ###################(utilizamos os modelos)
      if (categoria==4):
        # a probabilidade de dar fold vai variar em função da força da minha mao
        #print("######## Badugi ############")
        valores=retira_valores(mao_sem_parasitas)

        valor_deck=pesos_valor[valores[0]]+10*pesos_valor[valores[1]]+100*pesos_valor[valores[2]]+1000*pesos_valor[valores[3]]

        #print("valor_deck:",valor_deck)
        
        if(ronda==1):

          #Se tivermos na ronda 1 os parametros da prob de dar fold vao ter um certo peso
          #print("######## Ronda 1 ############")
          if(len(historicoJogador)==0):
            #se o jogador aind não apostou nada
            prob_fold=((apostaJogador/dinheiroJogador)*0.4)*(1-(valor_deck-1234)/9999)
           #print("prob_fold",prob_fold)
          else:
            #Caso já exista apostas na mesma ronda
            prob_fold=((apostaJogador/dinheiroJogador)*0.4+((sum(historicoJogador))/(2*len(historicoJogador)))*0.1)*(1-(valor_deck-1234)/9999)
           # print("prob_fold",prob_fold)
          
          #basicamente é um flip do fold na qual a prob é prob_fold
          dist_fold=tfd.Binomial(1.,probs=prob_fold)
          fold_samples = dist_fold.sample(1)
          fold_PC_list=fold_samples.numpy().tolist()
          fold_PC=fold_PC_list[0]
          #print("fold_PC:",fold_PC)

          if(fold_PC==1):
            print("O PC deu Fold!")
            apostaPC=apostaPC
            all_inPC=0
            foldPC=1
          
          else:
            if(len(historicoJogador)==0):
              p_allinPC = ((valor_deck- 1234)/9999 )*0.1 + 0.1
              #print("p_allinPC:",p_allinPC)
            else:
              p_allinPC = ((valor_deck- 1234)/9999 )*0.1 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.1
              #print("p_allinPC:",p_allinPC)

            dist_allin=tfd.Binomial(1.,probs=p_allinPC)
            allin_samples = dist_allin.sample(1)
            allin_PC_list=allin_samples.numpy().tolist()
            all_inPC=allin_PC_list[0]
            #print("all_inPC:",all_inPC)

            ## caso tenha feito all_in:
            if(all_inPC==1):
              print("O PC deu All_in!")
              apostaPC=dinheiroPC
              all_inPC=1
              foldPC=0
            ## caso não tenha dado all_in
            else:
              # agora vamos ver se vamos dar raise:
           
              p_raise=((valor_deck- 1234)/9999 )*0.4 +0.2
             # print("p_raise:",p_raise)

              dist_raise=tfd.Binomial(1.,probs=p_raise)
              raise_samples = dist_raise.sample(1)
              raisePC_list=raise_samples.numpy().tolist()
              raisePC=raisePC_list[0]
             # print("raisePC:",raisePC)

              if(raisePC==1):
                print("O PC deu Raise!")
                apostaPC=apostaPC+FaltaPagar + int(0.05*resto)
                all_inPC=0
                foldPC=0

              # se não der raise é porque deu call:
              else:
                print("O PC deu Call!")
                apostaPC=apostaPC+FaltaPagar 
                all_inPC=0
                foldPC=0
  

        elif(ronda==2):
          #Se tivermos na ronda 2 os parametros da prob de dar fold vao ter um certo peso
          #print("######## Ronda 2 ############")

          prob_fold=((apostaJogador/dinheiroJogador)*0.4 + ((sum(historicoJogador))/(2*len(historicoJogador)))*0.2 + ((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.1)*(1-(valor_deck-1234)/9999)
          #print("prob_fold",prob_fold)
          
          #basicamente é um flip do fold na qual a prob é prob_fold
          dist_fold=tfd.Binomial(1.,probs=prob_fold)
          fold_samples = dist_fold.sample(1)
          fold_PC_list=fold_samples.numpy().tolist()
          fold_PC=fold_PC_list[0]
          #print("fold_PC:",fold_PC)

          if(fold_PC==1):
            print("O PC deu Fold!")
            apostaPC=apostaPC
            all_inPC=0
            foldPC=1
          
          else:
            if(len(historicoJogador)==0):
               p_allinPC = ((valor_deck- 1234)/9999 )*0.1 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.1
               #print("p_allinPC:",p_allinPC)
            else:
               p_allinPC = ((valor_deck- 1234)/9999 )*0.1 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.05 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.05
               #print("p_allinPC:",p_allinPC)

            dist_allin=tfd.Binomial(1.,probs=p_allinPC)
            allin_samples = dist_allin.sample(1)
            allin_PC_list=allin_samples.numpy().tolist()
            all_inPC=allin_PC_list[0]
            #print("all_inPC:",all_inPC)

            ## caso tenha feito all_in:
            if(all_inPC==1):
              print("O PC deu All_in!")
              apostaPC=dinheiroPC
              all_inPC=1
              foldPC=0
            ## caso não tenha dado all_in
            else:
              # agora vamos ver se vamos dar raise:
           
              p_raise=((valor_deck- 1234)/9999 )*0.4 +0.2
              #print("p_raise:",p_raise)

              dist_raise=tfd.Binomial(1.,probs=p_raise)
              raise_samples = dist_raise.sample(1)
              raisePC_list=raise_samples.numpy().tolist()
              raisePC=raisePC_list[0]
              #print("raisePC:",raisePC)

              if(raisePC==1):
                print("O PC deu Raise!")
                apostaPC=apostaPC+FaltaPagar + int(0.05*resto)
                all_inPC=0
                foldPC=0

              # se não der raise é porque deu call:
              else:
                print("O PC deu Call!")
                apostaPC=apostaPC+FaltaPagar 
                all_inPC=0
                foldPC=0
        
        
        elif(ronda==3):
          #Se tivermos na ronda 3 os parametros da prob de dar fold vao ter um certo peso
          #print("######## Ronda 3 ############")

          prob_fold=((apostaJogador/dinheiroJogador)*0.4 + ((sum(historicoJogador))/(2*len(historicoJogador)))*0.2 + ((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.2)*(1-(valor_deck-1234)/9999)
          #print("prob_fold",prob_fold)
          
          #basicamente é um flip do fold na qual a prob é prob_fold
          dist_fold=tfd.Binomial(1.,probs=prob_fold)
          fold_samples = dist_fold.sample(1)
          fold_PC_list=fold_samples.numpy().tolist()
          fold_PC=fold_PC_list[0]
          #print("fold_PC:",fold_PC)

          if(fold_PC==1):
            print("O PC deu Fold!")
            apostaPC=apostaPC
            all_inPC=0
            foldPC=1
          
          else:
            if(len(historicoJogador)==0):
              p_allinPC = ((valor_deck- 1234)/9999 )*0.05 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.15
              #print("p_allinPC:",p_allinPC)
            else:
              p_allinPC = ((valor_deck- 1234)/9999 )*0.05 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.1 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.05
              #print("p_allinPC:",p_allinPC)

            dist_allin=tfd.Binomial(1.,probs=p_allinPC)
            allin_samples = dist_allin.sample(1)
            allin_PC_list=allin_samples.numpy().tolist()
            all_inPC=allin_PC_list[0]
            #print("all_inPC:",all_inPC)

            ## caso tenha feito all_in:
            if(all_inPC==1):
              print("O PC deu All_in!")
              apostaPC=dinheiroPC
              all_inPC=1
              foldPC=0
            ## caso não tenha dado all_in
            else:
              # agora vamos ver se vamos dar raise:
           
              p_raise=((valor_deck- 1234)/9999 )*0.4 +0.2
              #print("p_raise:",p_raise)

              dist_raise=tfd.Binomial(1.,probs=p_raise)
              raise_samples = dist_raise.sample(1)
              raisePC_list=raise_samples.numpy().tolist()
              raisePC=raisePC_list[0]
              #print("raisePC:",raisePC)

              if(raisePC==1):
                print("O PC deu Raise!")
                apostaPC=apostaPC+FaltaPagar + int(0.05*resto)
                all_inPC=0
                foldPC=0

              # se não der raise é porque deu call:
              else:
                print("O PC deu Call!")
                apostaPC=apostaPC+FaltaPagar 
                all_inPC=0
                foldPC=0
        

        elif(ronda==4):
          #Se tivermos na ronda 4 os parametros da prob de dar fold vao ter um certo peso
          #print("######## Ronda 4 ############")

          prob_fold=((apostaJogador/dinheiroJogador)*0.4 + ((sum(historicoJogador))/(2*len(historicoJogador)))*0.2 + ((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.2)*(1-(valor_deck-1234)/9999)
          #print("prob_fold",prob_fold)
          
          #basicamente é um flip do fold na qual a prob é prob_fold
          dist_fold=tfd.Binomial(1.,probs=prob_fold)
          fold_samples = dist_fold.sample(1)
          fold_PC_list=fold_samples.numpy().tolist()
          fold_PC=fold_PC_list[0]
          #print("fold_PC:",fold_PC)

          if(fold_PC==1):
            print("O PC deu Fold!")
            apostaPC=apostaPC
            all_inPC=0
            foldPC=1
          
          else:
            if(len(historicoJogador)==0):
              p_allinPC = ((valor_deck- 1234)/9999 )*0.05 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.15
              #print("p_allinPC:",p_allinPC)
            else:
              p_allinPC = ((valor_deck- 1234)/9999 )*0.05 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.1 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.05
              #print("p_allinPC:",p_allinPC)

            dist_allin=tfd.Binomial(1.,probs=p_allinPC)
            allin_samples = dist_allin.sample(1)
            allin_PC_list=allin_samples.numpy().tolist()
            all_inPC=allin_PC_list[0]
            #print("all_inPC:",all_inPC)

            ## caso tenha feito all_in:
            if(all_inPC==1):
              print("O PC deu All_in!")
              apostaPC=dinheiroPC
              all_inPC=1
              foldPC=0
            ## caso não tenha dado all_in
            else:
              # agora vamos ver se vamos dar raise:
           
              p_raise=((valor_deck- 1234)/9999 )*0.4 +0.2
              #print("p_raise:",p_raise)

              dist_raise=tfd.Binomial(1.,probs=p_raise)
              raise_samples = dist_raise.sample(1)
              raisePC_list=raise_samples.numpy().tolist()
              raisePC=raisePC_list[0]
              #print("raisePC:",raisePC)

              if(raisePC==1):
                print("O PC deu Raise!")
                apostaPC=apostaPC+FaltaPagar + int(0.05*resto)
                all_inPC=0
                foldPC=0

              # se não der raise é porque deu call:
              else:
                print("O PC deu Call!")
                apostaPC=apostaPC+FaltaPagar 
                all_inPC=0
                foldPC=0
        









      ################################ triplo ###################(utilizamos os modelos)
      if (categoria==3):
        # a probabilidade de dar fold vai variar em função da força da minha mao
       # print("######## Triplo ############")
        valores=retira_valores(mao_sem_parasitas)

        valor_deck=pesos_valor[valores[0]]+10*pesos_valor[valores[1]]+100*pesos_valor[valores[2]]

       # print("valor_deck:",valor_deck)
        
        if(ronda==1):

          #Se tivermos na ronda 1 os parametros da prob de dar fold vao ter um certo peso
         # print("######## Ronda 1 ############")
          if(len(historicoJogador)==0):
            #se o jogador aind não apostou nada
            prob_fold=((apostaJogador/dinheiroJogador)*0.6)+(1-(valor_deck-123)/1110)*0.1
            #print("prob_fold",prob_fold)

          else:
            #Caso já exista apostas na mesma ronda
            prob_fold=(apostaJogador/dinheiroJogador)*0.6+((sum(historicoJogador))/(2*len(historicoJogador)))*0.1 + (1-(valor_deck-123)/1110)*0.1
           # print("prob_fold",prob_fold)
          
          #basicamente é um flip do fold na qual a prob é prob_fold
          dist_fold=tfd.Binomial(1.,probs=prob_fold)
          fold_samples = dist_fold.sample(1)
          fold_PC_list=fold_samples.numpy().tolist()
          fold_PC=fold_PC_list[0]
         # print("fold_PC:",fold_PC)

          if(fold_PC==1):
            print("O PC deu Fold!")
            apostaPC=apostaPC
            all_inPC=0
            foldPC=1
          
          else:
            if(len(historicoJogador)==0):
              p_allinPC = ((valor_deck- 123)/1110 )*0.05 + 0.05
              #print("p_allinPC:",p_allinPC)
            else:
              p_allinPC = ((valor_deck- 123)/1110 )*0.05 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.05
             # print("p_allinPC:",p_allinPC)


            dist_allin=tfd.Binomial(1.,probs=p_allinPC)
            allin_samples = dist_allin.sample(1)
            allin_PC_list=allin_samples.numpy().tolist()
            all_inPC=allin_PC_list[0]
            #print("all_inPC:",all_inPC)

            ## caso tenha feito all_in:
            if(all_inPC==1):
              print("O PC deu All_in!")
              apostaPC=dinheiroPC
              all_inPC=1
              foldPC=0
            ## caso não tenha dado all_in
            else:
              # agora vamos ver se vamos dar raise:
           
              p_raise=((valor_deck- 123)/1110 )*0.2 +0.1
             # print("p_raise:",p_raise)

              dist_raise=tfd.Binomial(1.,probs=p_raise)
              raise_samples = dist_raise.sample(1)
              raisePC_list=raise_samples.numpy().tolist()
              raisePC=raisePC_list[0]
              #print("raisePC:",raisePC)

              if(raisePC==1):
                print("O PC deu Raise!")
                apostaPC=apostaPC+FaltaPagar + int(0.05*resto)
                all_inPC=0
                foldPC=0

              # se não der raise é porque deu call:
              else:
                print("O PC deu Call!")
                apostaPC=apostaPC+FaltaPagar 
                all_inPC=0
                foldPC=0
  

        elif(ronda==2):
          #Se tivermos na ronda 2 os parametros da prob de dar fold vao ter um certo peso
          #print("######## Ronda 2 ############")

          prob_fold=(apostaJogador/dinheiroJogador)*0.6+((sum(historicoJogador))/(2*len(historicoJogador)))*0.1+((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.1+(1-(valor_deck-123)/1110)*0.1
          #print("prob_fold",prob_fold)
          
          #basicamente é um flip do fold na qual a prob é prob_fold
          dist_fold=tfd.Binomial(1.,probs=prob_fold)
          fold_samples = dist_fold.sample(1)
          fold_PC_list=fold_samples.numpy().tolist()
          fold_PC=fold_PC_list[0]
          #print("fold_PC:",fold_PC)

          if(fold_PC==1):
            print("O PC deu Fold!")
            apostaPC=apostaPC
            all_inPC=0
            foldPC=1
          
          else:
            if(len(historicoJogador)==0):
              p_allinPC = ((valor_deck- 123)/1110 )*0.05 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.05
              #print("p_allinPC:",p_allinPC)
            else:
              p_allinPC = ((valor_deck- 123)/1110 )*0.05 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.025 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.025
             # print("p_allinPC:",p_allinPC)

            dist_allin=tfd.Binomial(1.,probs=p_allinPC)
            allin_samples = dist_allin.sample(1)
            allin_PC_list=allin_samples.numpy().tolist()
            all_inPC=allin_PC_list[0]
            #print("all_inPC:",all_inPC)

            ## caso tenha feito all_in:
            if(all_inPC==1):
              print("O PC deu All_in!")
              apostaPC=dinheiroPC
              all_inPC=1
              foldPC=0
            ## caso não tenha dado all_in
            else:
              # agora vamos ver se vamos dar raise:
           
              p_raise=((valor_deck- 123)/1110 )*0.2 +0.1
              #print("p_raise:",p_raise)

              dist_raise=tfd.Binomial(1.,probs=p_raise)
              raise_samples = dist_raise.sample(1)
              raisePC_list=raise_samples.numpy().tolist()
              raisePC=raisePC_list[0]
             # print("raisePC:",raisePC)

              if(raisePC==1):
                print("O PC deu Raise!")
                apostaPC=apostaPC+FaltaPagar + int(0.05*resto)
                all_inPC=0
                foldPC=0

              # se não der raise é porque deu call:
              else:
                print("O PC deu Call!")
                apostaPC=apostaPC+FaltaPagar 
                all_inPC=0
                foldPC=0
        
        
        elif(ronda==3):
          #Se tivermos na ronda 3 os parametros da prob de dar fold vao ter um certo peso
          #print("######## Ronda 3 ############")

          prob_fold=(apostaJogador/dinheiroJogador)*0.6+((sum(historicoJogador))/(2*len(historicoJogador)))*0.2+((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.1+(1-(valor_deck-123)/1110)*0.1
          #print("prob_fold",prob_fold)
          
          #basicamente é um flip do fold na qual a prob é prob_fold
          dist_fold=tfd.Binomial(1.,probs=prob_fold)
          fold_samples = dist_fold.sample(1)
          fold_PC_list=fold_samples.numpy().tolist()
          fold_PC=fold_PC_list[0]
          #print("fold_PC:",fold_PC)

          if(fold_PC==1):
            print("O PC deu Fold!")
            apostaPC=apostaPC
            all_inPC=0
            foldPC=1
          
          else:
            if(len(historicoJogador)==0):
              p_allinPC = ((valor_deck- 123)/1110 )*0.025 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.075
              #print("p_allinPC:",p_allinPC)
            else:
              p_allinPC = ((valor_deck- 123)/1110 )*0.025 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.05 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.025
              #print("p_allinPC:",p_allinPC)

            dist_allin=tfd.Binomial(1.,probs=p_allinPC)
            allin_samples = dist_allin.sample(1)
            allin_PC_list=allin_samples.numpy().tolist()
            all_inPC=allin_PC_list[0]
            #print("all_inPC:",all_inPC)

            ## caso tenha feito all_in:
            if(all_inPC==1):
              print("O PC deu All_in!")
              apostaPC=dinheiroPC
              all_inPC=1
              foldPC=0
            ## caso não tenha dado all_in
            else:
              # agora vamos ver se vamos dar raise:
           
              p_raise=((valor_deck- 123)/1110 )*0.2 +0.1
              #print("p_raise:",p_raise)

              dist_raise=tfd.Binomial(1.,probs=p_raise)
              raise_samples = dist_raise.sample(1)
              raisePC_list=raise_samples.numpy().tolist()
              raisePC=raisePC_list[0]
              #print("raisePC:",raisePC)

              if(raisePC==1):
                print("O PC deu Raise!")
                apostaPC=apostaPC+FaltaPagar + int(0.05*resto)
                all_inPC=0
                foldPC=0

              # se não der raise é porque deu call:
              else:
                print("O PC deu Call!")
                apostaPC=apostaPC+FaltaPagar 
                all_inPC=0
                foldPC=0
        

        elif(ronda==4):
          #Se tivermos na ronda 4 os parametros da prob de dar fold vao ter um certo peso
          #print("######## Ronda 4 ############")

          prob_fold=(apostaJogador/dinheiroJogador)*0.6+((sum(historicoJogador))/(2*len(historicoJogador)))*0.2+((1-sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.1+(1-(valor_deck-123)/1110)*0.1
          #print("prob_fold",prob_fold)
          
          #basicamente é um flip do fold na qual a prob é prob_fold
          dist_fold=tfd.Binomial(1.,probs=prob_fold)
          fold_samples = dist_fold.sample(1)
          fold_PC_list=fold_samples.numpy().tolist()
          fold_PC=fold_PC_list[0]
          #print("fold_PC:",fold_PC)

          if(fold_PC==1):
            print("O PC deu Fold!")
            apostaPC=apostaPC
            all_inPC=0
            foldPC=1
          
          else:
            if(len(historicoJogador)==0):
              p_allinPC = ((valor_deck- 123)/1110 )*0.025 + ((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.075
              #print("p_allinPC:",p_allinPC)
            else:
              p_allinPC = ((valor_deck- 123)/1110 )*0.025 + (1-(sum(historicoJogador))/(2*len(historicoJogador)))*0.05 +((sum(CartasRetiradas))/(2*len(CartasRetiradas)))*0.025
              #print("p_allinPC:",p_allinPC)

            dist_allin=tfd.Binomial(1.,probs=p_allinPC)
            allin_samples = dist_allin.sample(1)
            allin_PC_list=allin_samples.numpy().tolist()
            all_inPC=allin_PC_list[0]
           # print("all_inPC:",all_inPC)

            ## caso tenha feito all_in:
            if(all_inPC==1):
              print("O PC deu All_in!")
              apostaPC=dinheiroPC
              all_inPC=1
              foldPC=0
            ## caso não tenha dado all_in
            else:
              # agora vamos ver se vamos dar raise:
           
              p_raise=((valor_deck- 123)/1110 )*0.2 +0.1
              #print("p_raise:",p_raise)

              dist_raise=tfd.Binomial(1.,probs=p_raise)
              raise_samples = dist_raise.sample(1)
              raisePC_list=raise_samples.numpy().tolist()
              raisePC=raisePC_list[0]
              #print("raisePC:",raisePC)

              if(raisePC==1):
                print("O PC deu Raise!")
                apostaPC=apostaPC+FaltaPagar + int(0.05*resto)
                all_inPC=0
                foldPC=0

              # se não der raise é porque deu call:
              else:
                print("O PC deu Call!")
                apostaPC=apostaPC+FaltaPagar 
                all_inPC=0
                foldPC=0
        


      ######################### duplo ####################
      if(categoria==2):
        #print("### Duplo ####")

        if(ronda==1):
         # print("rodada 1")
          prob_fold=0.7
          p_raise=0.05
        elif(ronda==2):
          #print("rodada 2")
          prob_fold=0.8
          p_raise=0.03
        elif(ronda==3):
          #print("rodada 3")
          prob_fold=0.95
          p_raise=0.02
        elif(ronda==4):
         # print("rodada 4")
          prob_fold=1
          p_raise=0.02

        

        #print("prob_fold:",prob_fold)
        #print("p_raise:",p_raise)

        dist_fold=tfd.Binomial(1.,probs=prob_fold)
        fold_samples = dist_fold.sample(1)
        fold_PC_list=fold_samples.numpy().tolist()
        fold_PC=fold_PC_list[0]
       # print("fold_PC:",fold_PC)

        if(fold_PC==1):
          print("O PC deu Fold!")
          apostaPC=apostaPC
          all_inPC=0
          foldPC=1
        else:
          # dar all_in é estúpido aqui, daí consideramos que é 0 a probabilidade

          # então só pode dar raise ou call

          

          dist_raise=tfd.Binomial(1.,probs=p_raise)
          raise_samples = dist_raise.sample(1)
          raisePC_list=raise_samples.numpy().tolist()
          raisePC=raisePC_list[0]
          #print("raisePC:",raisePC)

          if(raisePC==1):
            print("O PC deu Raise!")
            apostaPC=apostaPC+FaltaPagar+ int(0.05*resto)
            all_inPC=0
            foldPC=0
          
          else:
            print("O PC deu Call!")
            apostaPC=apostaPC+FaltaPagar
            all_inPC=0
            foldPC=0






      ######################### solo ####################
      if(categoria==1):
        #print("### Solo ####")

        if(ronda==1):
          #print("rodada 1")
          prob_fold=0.8
          p_raise=0.03
        elif(ronda==2):
         # print("rodada 2")
          prob_fold=0.9
          p_raise=0.02
        elif(ronda==3):
         # print("rodada 3")
          prob_fold=1
          p_raise=0.01
        elif(ronda==4):
          #print("rodada 4")
          prob_fold=1
          p_raise=0.02

        #print("prob_fold:",prob_fold)
       # print("p_raise:",p_raise)

        

        dist_fold=tfd.Binomial(1.,probs=prob_fold)
        fold_samples = dist_fold.sample(1)
        fold_PC_list=fold_samples.numpy().tolist()
        fold_PC=fold_PC_list[0]
        #print("fold_PC:",fold_PC)

        if(fold_PC==1):
          print("O PC deu Fold!")
          apostaPC=apostaPC
          all_inPC=0
          foldPC=1
        else:
          # dar all_in é estúpido aqui, daí consideramos que é 0 a probabilidade

          # então só pode dar raise ou call

          

          dist_raise=tfd.Binomial(1.,probs=p_raise)
          raise_samples = dist_raise.sample(1)
          raisePC_list=raise_samples.numpy().tolist()
          raisePC=raisePC_list[0]
          #print("raisePC:",raisePC)

          if(raisePC==1):
            print("O PC deu Raise!")
            apostaPC=apostaPC+FaltaPagar+ int(0.05*resto)
            all_inPC=0
            foldPC=0
          
          else:
            print("O PC deu Call!")
            apostaPC=apostaPC+FaltaPagar
            all_inPC=0
            foldPC=0


        


  return [apostaPC,all_inPC,foldPC]




In [3]:
#########################################################################################################################
########################################### Aposta e InputApostaJogador ######################################################################
#########################################################################################################################

###################### INPUT_APOSTA_JOGADOR #########################


# precisa de receber também o histórico do jogador para poder gravar o historial de jogadas
def inputApostaJogador(apostaPC,apostaJogador,dinheiroJogador,historicoJogador):
  
  #resto do dinheiro que o jogador tem
  resto = dinheiroJogador - apostaJogador

  #Dinheiro que falta para o jogador igualar a aposta do PC
  FaltaPagar = apostaPC - apostaJogador
  
  respostaValida = 0

  while(not(respostaValida)):

    if(FaltaPagar>=resto):
      #o jogador nao tem dinheiro suficiente para pagar => tem de ir all in se quer jogar
      # caso contrario dá fold

      print('Para continuar no jogo tem de dar all in.\n1 = dar all in\n0 = fold')
      decisao = int(input('Insira a sua opcao: '))

      if(decisao):
        #quer ir a jogo e terá que dar all in 

        apostaJogador = dinheiroJogador
        all_inJogador = 1
        foldJogador = 0
        respostaValida = 1
      
      else:
        # é porque desistiu e deu fold

        #apostaJogador = apostaJogador
        all_inJogador = 0
        foldJogador = 1
        respostaValida = 1

    else:
      # FaltaPagar<resto

      #Verificar se pretende dar fold de imediato
      print("\nPretende dar fold?\nSe sim, digite 1\nSe não, digite 0\n")
      foldJogador = int(input())

      if(foldJogador):

        apostaJogador = apostaJogador
        all_inJogador = 0
        #foldJogador = 1 - ja esta feito ao fazer input
        respostaValida = 1

      else:
        print("O valor da sua aposta anterior é:",apostaJogador)
        print("O valor da aposta do PC vai em:",apostaPC)
        print('Digite o valor inteiro que pretende acrescentar à sua aposta.\nSe pretender dar check insira o valor 0.\n')
        print('Dinheiro disponivel: ',dinheiroJogador - apostaJogador)
        aumento = int(input('Aumento da aposta = '))

        if((aumento>=FaltaPagar) & (aumento<resto)):
          #vamos dar raise ou call ou check
          
          # se deu check:
          if(aumento==0):
            historicoJogador.append(0)
          
          # se deu call:
          elif(aumento==FaltaPagar):
            historicoJogador.append(1)
          
          # se deu raise:
          else:
            historicoJogador.append(2)
          

          apostaJogador = apostaJogador + aumento
          foldJogador = 0
          all_inJogador = 0
          respostaValida = 1

        elif((aumento>=FaltaPagar) & (aumento == resto)):
          # esta a dar all in
          apostaJogador = apostaJogador + aumento
          all_inJogador = 1
          foldJogador = 0
          respostaValida = 1

        else:
          # Entra aqui nos casos:
          #   -> aumento < FaltaPagar & aumento < resto
          #   -> aumento >= FaltaPagar & aumento > resto
          print('Resposta invalida!\nPossiveis hipoteses:\n1 - Nao tem dinheiro para pagar\n2 - o aumento foi demasiado baixo\nTente novamente\n3 - deu check e não podia')


      

  return [apostaJogador, foldJogador ,all_inJogador,historicoJogador]










####################### APOSTA #######################################



def Aposta(ronda,SmallBlind, BigBlind, all_in, pote, JogadorBigBlind,dinheiroPC,dinheiroJogador,maoPC,pesosValor,historicoJogador,CartasRetiradas,maoJogador):

#Nota: o all_in que vem para a primeira aposta, vem all_in=0(tem de ser declarado na funcao jogo)


#Nota imp: imaginemos que o jogador tem 1000$ e o PC 800$
#          Imaginemos que o jogador dá all in => apostou 1000. Se o PC quer continuar a jogar,
#         tem obrigatoriamente de dar all in tambem apesar de ter menos dinheiro.
#          Imaginemos que o PC ganha porque tem melhor mao. Então, no fim os valores ficam:
#          jogador = 200(1000-800) e PC = 1600(800+800)


  

  #Se já vem um all in detras, entao não queremos fazer nada
  if(not(all_in)):

    #ver se estamos na 1 ronda
    if(ronda==1):
      # na primeira ronda temos de atribuir a big e a small blind
 
      #Como estamos no inicio da primeira ronda ainda nenhum deu all_in, nem fold
      all_inJogador = 0
      all_inPC = 0
      foldPC = 0
      foldJogador = 0
      aux = 1

      if(JogadorBigBlind == 0):
        # o PC tem a big blind
        print("\n")
        print("O PC tem a Big Blind!")
        print("O Jogador tem a Small Blind!")
        print("Quem começa é o Jogador!")
        print("\n")

        apostaJogador = SmallBlind
        apostaPC = BigBlind

      else: 
        # o jogador tem a big blind
        print("\n")
        print("O Jogador tem a Big Blind!")
        print("O PC tem a Small Blind!")
        print("Quem começa é o PC!")
        print("\n")

        print("\n")
        apostaJogador = BigBlind
        apostaPC = SmallBlind
  
    else:
      # ronda > 1 => as blinds já não interessam
      print("\n")
      print("Esta é uma ronda diferente de 1, logo quem começa é quem foi selecionado como a Small Blind!")
      print("\n")
      all_inJogador = 0
      all_inPC = 0
      foldPC = 0
      foldJogador = 0
      aux = 1

      apostaJogador = 0
      apostaPC = 0
      
    print("################# Ronda nº:" + str(ronda) + " ######################")
    #o que estava antes:((apostaPC!=apostaJogador)|aux) & (not(foldPC)) & (not(foldJogador)) & (not(all_inJogador)) & (not(all_inPC))
    while(((apostaPC!=apostaJogador)|aux) and (not(foldPC)) and (not(foldJogador))):
      

      
      aux = 0 # => ((apostaPC!=apostaJogador)|0) = (apostaPC!=apostaJogador)
      
      #Saimos do ciclo while quando
      # -> as apostas sao iguais
      # -> um deles deu fold
      # -> um deu all in e o outro call
      
      
      if(JogadorBigBlind == 0):
        # se ambos derem all_in, então acaba a ronda de apostas!!
        if((all_inPC==1) and (all_inJogador==1)):
          print('\n')
          print("Ambos os participantes deram All in!!")
          print('\n')
          break

        #o PC tem a big blind quem começa a jogar é o oponente(tem a small blind)
        print("É a vez do Jogador:")
        print("Mão do Jogador:",maoJogador)
        apostaJogadortotal = inputApostaJogador(apostaPC,apostaJogador,dinheiroJogador,historicoJogador)
       
        apostaJogador = apostaJogadortotal[0] # se o jogador der all_in, nao basta retornar all_inJogador=1, mas tambem apostaJogador = dinheiroJogador
                                         # se o jogador der fold, este apostaJogador vai ser = ao apostaJogador que serviu de input a inputApostaJogador(...apostaJogador...)
        foldJogador = apostaJogadortotal[1]
        all_inJogador = apostaJogadortotal[2] # se dinheiroPC >= dinheiroJogador e all_inPC = 1 => automaticamente all_inJogador terá que ser 1
        historicoJogador=apostaJogadortotal[3]
        print("\n\n")
        print("Aposta do Jogador:",apostaJogador)
        print("\n")
        #print("O jogador deu Fold?\n sim-->1\n não-->0\nResposta:",foldJogador)
        #print("\n")
        #print("O jogador deu All_in?\n sim-->1\n não-->0\nResposta:",all_inJogador)
        #print("\n")
        #print("Histórico de Jogadas do jogador:\n Raise-->2\nCall-->1\nCheck-->0\nResposta:",historicoJogador)
        #print("\n\n")

        if(foldJogador == 1):
          break


        if(all_inPC==1):
          break
        # funçao Aposta:
        #input: ronda,SmallBlind, BigBlind, all_in, pote, JogadorBigBlind,dinheiroPC,dinheiroJogador,maoPC,pesosValor,historicoJogador,CartasRetiradas

        #função ApostaPC
        #input: apostaPC,apostaJogador,dinheiroPC,dinheiroJogador,ronda,maoPC,pesosValor,historicoJogador,CartasRetiradas
        #output: apostaPC,all_inPC,foldPC
      
        print("É a vez do PC:")
        apostaPCtotal = ApostaPC(apostaPC,apostaJogador,dinheiroPC,dinheiroJogador,ronda,maoPC,pesosValor,historicoJogador,CartasRetiradas)

        apostaPC = apostaPCtotal[0]# se o PC der all_in, nao basta retornar all_inPC=1, mas tambem apostaPC = dinheiroPC 
        foldPC = apostaPCtotal[2]
        all_inPC = apostaPCtotal[1]# se dinheiroPC <= dinheiroJogador e all_inJogador = 1 => automaticamente all_inPC terá que ser 1

        print("\n\n")
        print("Aposta do PC:",apostaPC)
        print("\n")
        #print("O PC deu Fold?\n sim-->1\n não-->0\nResposta:",foldPC)
        #print("\n")
        #print("O PC deu All_in?\n sim-->1\n não-->0\nResposta:",all_inPC)
        #print("\n\n")

        if(all_inJogador==1):
          break
        


      else:

        # se ambos derem all_in, então acaba a ronda de apostas!!
        if((all_inPC==1) and (all_inJogador==1)):
          print('\n')
          print("Ambos os participantes deram All in!!")
          print('\n')
          break
       

        print("É a vez do PC:")
        apostaPCtotal = ApostaPC(apostaPC,apostaJogador,dinheiroPC,dinheiroJogador,ronda,maoPC,pesosValor,historicoJogador,CartasRetiradas)

        apostaPC = apostaPCtotal[0]# se o PC der all_in, nao basta retornar all_inPC=1, mas tambem apostaPC = dinheiroPC 
        foldPC = apostaPCtotal[2]
        all_inPC = apostaPCtotal[1]# se dinheiroPC <= dinheiroJogador e all_inJogador = 1 => automaticamente all_inPC terá que ser 1
        print("\n\n")
        print("Aposta do PC:",apostaPC)
        print("\n")
        #print("O PC deu Fold?\n sim-->1\n não-->0\nResposta:",foldPC)
        #print("\n")
        #print("O PC deu All_in?\n sim-->1\n não-->0\nResposta:",all_inPC)
        #print("\n\n")



        if(foldPC == 1):
          
          break
        if(all_inJogador==1):
          break

        print("É a vez do Jogador:")
        print("Mão do Jogador:",maoJogador)
        # vez do jogador decidir o que vai apostar
        apostaJogadortotal = inputApostaJogador(apostaPC,apostaJogador,dinheiroJogador,historicoJogador)
        apostaJogador = apostaJogadortotal[0] 
        foldJogador = apostaJogadortotal[1]
        all_inJogador = apostaJogadortotal[2]
        historicoJogador=apostaJogadortotal[3]
        print("\n\n")
        print("Aposta do Jogador:",apostaJogador)
        print("\n")
       # print("O jogador deu Fold?\n sim-->1\n não-->0\nResposta:",foldJogador)
       # print("\n")
       # print("O jogador deu All_in?\n sim-->1\n não-->0\nResposta:",all_inJogador)
       # print("\n")
       # print("Histórico de Jogadas do jogador:\n Raise-->2\nCall-->1\nCheck-->0\nResposta:",historicoJogador)
       # print("\n\n")

        if(all_inPC==1):
          break

    # Quando saimos do ciclo while podemos ter os seguintes casos:
    """
    -> alguem deu fold
    -> ambos igualaram as apostas(ambos podem ter dado check => apostaJogador=apostaPC=0)
    -> alguem deu all_in e o outro call
    """

    # Se se ambos forem 0, nenhum deu fold(=> fold=0). Se um deles der fold => fold=1
    fold = foldPC | foldJogador

    if(fold):
      # se alguem deu fold, acaba já tudo
      pote = pote+apostaPC + apostaJogador
      dinheiroPC = dinheiroPC - apostaPC
      dinheiroJogador = dinheiroJogador - apostaJogador

      # se algum deles der fold o outro fica com o dinheiro
      if(foldPC==1):
        print('\n')
        print("Como o PC deu Fold, o jogo termina e quem ganha o pote é o Jogador!")
        print('\n')
        #O PC deu fold => o jogador fica com o pote
        dinheiroJogador = dinheiroJogador + pote
        print("Mão do PC: ",maoPC)
        print("Mão do Jogador: ",maoJogador)
        print("Dinheiro do Jogador: ",dinheiroJogador)
        print("Dinheiro do PC: ",dinheiroPC)
        
        pote = 0

      elif(foldJogador==1):
        print('\n')
        print("Como o Jogador deu Fold, o jogo termina e quem ganha o pote é o PC!")
        print('\n')
        # o Jogador deu fold => o PC fica com o pote
        dinheiroPC = dinheiroPC + pote
        print("Mão do PC: ",maoPC)
        print("Mão do Jogador: ",maoJogador)
        print("Dinheiro do Jogador: ",dinheiroJogador)
        print("Dinheiro do PC: ",dinheiroPC)
        pote = 0


    else: 
      #entrar aqui significa que ninguem deu fold

      if(all_inJogador | all_inPC):
        # alguem ou ambos deram all in

        all_in = all_inJogador | all_inPC

        if((all_inJogador==1) & (all_inPC==1) & (dinheiroJogador>=dinheiroPC)):
          print('\n')
          print("Ambos deram All in, porém o Jogador tem mais/ou igual dinheiro que o PC! No Jogo do Badugi isso obriga o Jogador a fazer Call á aposta do PC!")
          print('\n')
          #Se o jogador tem mais dinheiro que o PC e fez all in, obriga o PC a ir all in
          #uma vez que o jogador tem mais dinheiro que o pc é como se o PC tivesse dado all in e o jogador desse call
          apostaPC = dinheiroPC
          dinheiroPC = 0
          apostaJogador = apostaPC
          dinheiroJogador = dinheiroJogador - apostaJogador
          all_inJogador = 0
          pote = pote + apostaPC + apostaJogador

        elif((all_inJogador==0) & (all_inPC==1) & (dinheiroJogador>=dinheiroPC)):
          print('\n')
          print("O PC deu All in e o Jogador não, porém o Jogador tem mais/ou igual dinheiro que o PC! No jogo do Badugi isso obriga o Jogador a fazer Call à aposta do PC!")
          print('\n')
          apostaPC = dinheiroPC
          dinheiroPC = 0
          apostaJogador = apostaPC
          dinheiroJogador = dinheiroJogador - apostaJogador
          pote = apostaPC + apostaJogador

        elif((all_inJogador==1) & (all_inPC==1) & (dinheiroJogador<=dinheiroPC)):
          print('\n')
          print("Ambos deram All in, porém o PC tem mais/ou igual dinheiro que o Jogador! No Jogo do Badugi isso obriga o PC a fazer Call à aposta do Jogador!")
          print('\n')
          apostaJogador = dinheiroJogador
          dinheiroJogador = 0
          apostaPC = apostaJogador
          dinheiroPC = dinheiroPC - apostaPC
          all_inPC = 0
          pote = pote + apostaPC + apostaJogador

        else:
          # aqui é caso: (all_inJogador==1) & (all_inPC==0) & dinheiroJogador<=dinheiroPC
          print('\n')
          print("O Jogador deu All in e o PC não, porém o PC tem mais/ou igual dinheiro que o Jogador! No jogo do Badugi isso obriga o PC a fazer Call à aposta do Jogador!")
          print('\n')
          apostaJogador = dinheiroJogador
          dinheiroJogador = 0
          apostaPC = apostaJogador
          dinheiroPC = dinheiroPC - apostaPC
          pote = pote + apostaPC + apostaJogador

      else:
        # é porque ninguem deu all_in nem fold. Ou ambos deram check(apostaJogador=apostaPC=0) ou apostaram qualquer valor(apostaJogador=apostaPC)
        pote = pote + apostaPC + apostaJogador
        dinheiroPC = dinheiroPC - apostaPC
        dinheiroJogador = dinheiroJogador - apostaJogador


  #######
  else:
    fold=0
    print("Um dos participantes deu All in, como tal a ronda de apostas fica sem efeito!")
    #print(" Valor do pote: ",pote)
    print("Dinheiro do Jogador: ",dinheiroJogador)
    print("Dinheiro do PC: ",dinheiroPC)
    # quando temos all_in nao fazemos nada e passamos para o proximo draw
    
     

  #a ronda de apostas seguinte sera 2
  ronda += 1

  return [ronda, all_in, JogadorBigBlind,fold,pote,dinheiroPC,dinheiroJogador,historicoJogador]



pesos_valor={
    'A':13,
    '2':12,
    '3':11,
    '4':10,
    '5':9,#----------------------------------------------------------------------------> joao: Isto não está repetido 2 vezes? Na função jogo já tens la dentro isto
    '6':8,
    '7':7,
    '8':6,
    '9':5,
    '10':4,
    'D':3,
    'V':2,
    'R':1   

}


# input do Aposta:ronda,SmallBlind, BigBlind, all_in, pote, JogadorBigBlind,dinheiroPC,dinheiroJogador,maoPC,pesosValor,historicoJogador,CartasRetiradas
# output do Aposta:ronda, all_in, JogadorBigBlind,fold,pote,dinheiroPC,dinheiroJogador,historicoJogador 

#JogadorBigBlind=0---> O PC tem a Big Blind e o jogador começa
#JogadorBigBlind=1---> O PC tem a small Blind e o PC começa

In [4]:
#############################################################################################################################
################################################### FUNÇÃO JOGO ##############################################################
#############################################################################################################################

def Jogo(dinheiroPC,dinheiroJogador,JogadorBigBlind,SmallBlind,BigBlind):
  pesosValor={
    'A':13,
    '2':12,
    '3':11,
    '4':10,
    '5':9,
    '6':8,
    '7':7,
    '8':6,
    '9':5,
    '10':4,
    'D':3,
    'V':2,
    'R':1   }
  
  baralho= ['2C','2O','2E','2P','3C','3O','3E','3P','4C','4O','4E','4P','5C','5O','5E','5P','6C','6O','6E','6P','7C','7O','7E','7P','8C','8O','8E','8P','9C','9O','9E','9P','10C','10O','10E','10P','DC','DO','DE','DP','VC','VO','VE','VP','RC','RO','RE','RP','AC','AO','AE','AP']

  # primeiro sorteio(tirar 4 cartas para o PC) = [maoPC, baralho sem 4 cartas]
  baralho2=baralho.copy()
  primeiro_sorteio = retira_n_cartas(baralho2,4)
  mao_PC = primeiro_sorteio[0]
  cartas_possiveis = primeiro_sorteio[1] # sem as 4 cartas

  cartas_possiveis2=cartas_possiveis.copy()

  # segundo sorteio(tirar 4 cartas para o jogador) = [maoJogador, baralho sem 8 cartas]
  segundo_sorteio = retira_n_cartas(cartas_possiveis2,4)
  mao_Jogador = segundo_sorteio[0]
  baralho_atualizado = segundo_sorteio[1] # baralho sem 8 cartas


  # neste momento temos:
  # -- baralho completo (baralho)
  # -- baralho das cartas possíveis para o PC (cartas_possiveis)
  # -- baralho verdadeiro (baralho_atualizado)


  ronda = 1
  #array que guarda todas as jogadas do oponente, se deu check, raise, call...
  JogadasAntOponente = []

  #lista do número de cartas retiradas pelo jogador
  CartasRetiradas=[]

  all_in = 0
  pote = 0
 

  #JogadorBigBlind=0---> O PC tem a Big Blind e o jogador começa a apostar
  #JogadorBigBlind=1---> O PC tem a small Blind e o PC começa
  
  #input: ronda,SmallBlind, BigBlind, all_in, pote, JogadorBigBlind,dinheiroPC,dinheiroJogador,maoPC,pesosValor,historicoJogador,CartasRetiradas,maoJogador
  a = Aposta(ronda,SmallBlind,BigBlind,all_in,pote,JogadorBigBlind,dinheiroPC,dinheiroJogador,mao_PC,pesosValor,JogadasAntOponente,CartasRetiradas,mao_Jogador) 
  # return da funcao Aposta: [Proximaronda, all_in, JogadorBigBlind,fold,pote,dinheiroPC,dinheiroJogador,historicoJogador]
  
  # atualizar o historico do oponente:
  JogadasAntOponente=a[7]
  print("\n")
  print("Valor do pote: ",a[4])
  print("\n")
  #Se alguem deu fold => fold=1
  fold = a[3]

  if(fold==1):
    #alguem deu fold => acaba logo o jogo
    return [a[5],a[6],mao_PC,mao_Jogador] #[dinheiroPC,dinheiroJogador]
  else:
    ####### ronda de apostas #############
    
    cartasDescartadasPC = []
    aux = mao_PC.copy()
    
    #print("aux:",aux)
    #CartasConhecidas = cartas da mao + as que descartei
    #CartasConhecidas = aux.append(cartasDescartadasPC)
    CartasConhecidas=aux+cartasDescartadasPC

    baralho3=baralho.copy() ##### alterei pois se tirarem cartas do baralho principal, vão perder o principal e acho que não querem que isso aconteça
    #print("baralho:",baralho)
   # print("baralho3:",baralho3)
   # print("cartas conhecidas:",CartasConhecidas)

    #As cartas que me podem sair sao: CartasPossiveis = baralho - CartasConhecidas
    # a função tira remove do baralho completo as cartas que já conheces e retorna o baralho sem essas cartas
    Cartaspossiveis = tira(baralho3,CartasConhecidas)
    
    #print("Cartaspossivies:",Cartaspossiveis)
    # O draw permite o Pc trocar cartas e o jogador tambem
   
    ## ATENÇÂO: é preciso que a função draw retorne também uma lista que me diga se o jogador trocou ou não carta, se trocou uma e a lista de input for [], então deve retornar [1], se não trocar retorna [0]
    # basicamente é fazer um append na lista de input. Preciso disso para o Aposta
    # a lista que falo chama-se "CartasRetiradas" e já coloquei no input da draw e modifiquei a drawJogador para cumprir isso

    ## ATENÇÂO: Modifiquei o parâmetro "baralho" da draw, para "baralho-atualizado"
    mao_PC.sort(key=retorna_dic_1)
    
    #print("input da out!!!!!!!!!!!!!!")
    print("Mão do Jogador: ",mao_Jogador)
    #print("mao_PC:",mao_PC)
    #print("cartasDescartadasPC:",cartasDescartadasPC)
    #print("baralho_atualizado:",baralho_atualizado)
    #print("Cartaspossiveis:",Cartaspossiveis)
    #print("CartasRetiradas:",CartasRetiradas)

    out = draw(mao_Jogador,mao_PC,cartasDescartadasPC,baralho_atualizado,Cartaspossiveis,CartasRetiradas)
    #output
    #out = [cartasJogador,CartasPC, CartasDescartas, baralho , cartaspossiveis,CartasRetiradas] --------> estas cartaspossieveis são: baralho-CartasConhecidas(cartasDescartadas + nossa mao)?
    #out=[cartasJogador,baralho_jogador,cartasRetiradas,CartasPC, CartasDescartas, baralho , cartaspossiveis]
    

    
   # print("output da draw!!!!!!!!!!!!!!!!")
    #print("Nº de cartas trocadas pelo PC: ",len(out[4]))
    print("Mão do Jogador: ",out[0])
    #print("baralho_jogador:",out[1])
    #print("cartasRetiradas:",out[2])
    #print("Mao_PC:",out[3])
    #print("CartasDescartadas:",out[4])
    #print("baralho:",out[5])
    #print("cartaspossiveis:",out[6])
    
    CartasRetiradas=out[2]



    #--------------------2ªaposta--------------------------------------------

    
    
    # input do Aposta:ronda,SmallBlind, BigBlind, all_in, pote, JogadorBigBlind,dinheiroPC,dinheiroJogador,maoPC,pesosValor,historicoJogador,CartasRetiradas
    # output do Aposta:proximaronda, all_in, JogadorBigBlind,fold,pote,dinheiroPC,dinheiroJogador,historicoJogador 
    a2 = Aposta(a[0],SmallBlind,BigBlind,a[1],a[4],a[2],a[5],a[6],out[3],pesosValor,a[7],CartasRetiradas,out[0]) 
    print("\n")
    print("Valor do pote: ",a2[4])
    print("\n")

    if(a2[3]==1):
      #se alguem deu fold acaba já
      return [a2[5],a2[6],out[3],out[0]]
    else:
      cartasDescartadasPC2 = out[4]

      mao_PCaux=out[3].copy()

      #aux3 = mao_PCaux.append(cartasDescartadasPC2)
      aux3=mao_PCaux + cartasDescartadasPC2
      baralho4=baralho.copy()
      Cartaspossiveis2 = tira(baralho4,aux3) # remove do baralho as cartas que já conheço

      #output:
      #out2 = [cartasJogador,CartasPC, CartasDescartas, baralho , cartaspossiveis,CartasRetiradas] --------> estas cartaspossieveis são: baralho-CartasConhecidas(cartasDescartadas + nossa mao)?
      #out2=[cartasJogador,baralho_jogador,cartasRetiradas,CartasPC, CartasDescartas, baralho , cartaspossiveis]

      mao_PC=out[3]
      mao_PC.sort(key=retorna_dic_1)

      #print("input da out!!!!!!!!!!!!!!")
      print("Mão do Jogador:",out[0])
      #print("mao_PC:",mao_PC)
      #print("cartasDescartadasPC:",out[4])
      #print("baralho_atualizado:",out[5])
      #print("Cartaspossiveis:",out[6])
      #print("CartasRetiradas:",out[2])


      out2= draw(out[0],out[3],cartasDescartadasPC2,out[5],Cartaspossiveis2,CartasRetiradas)

      #print("output da draw!!!!!!!!!!!!!!!!")
      #print("mao_Jogador:",out2[0])
     # print("Nº de cartas trocadas pelo PC: ",len(out2[4]))
      print("Mão do Jogador: ",out2[0])
      #print("cartasRetiradas:",out2[2])
      #print("Mao_PC:",out2[3])
      #print("CartasDescartadas:",out2[4])
      #print("baralho:",out2[5])
      #print("cartaspossiveis:",out2[6])

      CartasRetiradas=out2[2]

      #----------------------3ªaposta-------------------------------------------------

      # input do Aposta:ronda,SmallBlind, BigBlind, all_in, pote, JogadorBigBlind,dinheiroPC,dinheiroJogador,maoPC,pesosValor,historicoJogador,CartasRetiradas
      # output do Aposta:proximaronda, all_in, JogadorBigBlind,fold,pote,dinheiroPC,dinheiroJogador,historicoJogador
      a3 = Aposta(a2[0],SmallBlind,BigBlind,a2[1],a2[4],a2[2],a2[5],a2[6],out2[3],pesosValor,a2[7],CartasRetiradas,out2[0])
      #Aposta(a[0],SmallBlind,BigBlind,a[1],a[4],a[2],a[5],a[6],out[3],pesosValor,JogadasAntOponente,CartasRetiradas) 
      print("\n")
      print("Valor do pote: ",a3[4])
      print("\n")
      if(a3[3]==1):
        return [a3[5],a3[6],out2[3],out2[0]]
        #output: [dinheiroPC,dinheiroJogador,maoPC,maoJogador]
        
      else:
        cartasDescartadasPC3 = out2[4]
        mao_PCaux=out2[3].copy()
        #aux4 = mao_PCaux.append(cartasDescartadasPC3)
        aux4=mao_PCaux+cartasDescartadasPC3
        baralho5=baralho.copy()
        Cartaspossiveis3 = tira(baralho5,aux4)
        
        #input: mao_Jogador,mao_PC,cartasDescartadasPC,baralho_atualizado,Cartaspossiveis,CartasRetiradas

        mao_PC=out2[3]
        mao_PC.sort(key=retorna_dic_1)

        #print("input da out!!!!!!!!!!!!!!")
        print("Mão do Jogador :",out2[0])
        #print("mao_PC:",mao_PC)
        #print("cartasDescartadasPC:",out2[4])
        #print("baralho_atualizado:",out2[5])
        #print("Cartaspossiveis:",out2[6])
        #print("CartasRetiradas:",out2[2])



        out3 = draw(out2[0],out2[3],cartasDescartadasPC3,out2[5],Cartaspossiveis3,CartasRetiradas)

    #    print("output da draw!!!!!!!!!!!!!!!!")
        #print("Nº de cartas trocadas pelo PC: ",len(out3[4]))
        print("Mão do Jogador :",out3[0])
     #   print("baralho_jogador:",out3[1])
     #   print("cartasRetiradas:",out3[2])
     #   print("Mao_PC:",out3[3])
     #   print("CartasDescartadas:",out3[4])
      #  print("baralho:",out3[5])
      #  print("cartaspossiveis:",out3[6])

        #output:
        #out2 = [cartasJogador,CartasPC, CartasDescartas, baralho , cartaspossiveis,CartasRetiradas] --------> estas cartaspossieveis são: baralho-CartasConhecidas(cartasDescartadas + nossa mao)?
        #out2=[cartasJogador,baralho_jogador,cartasRetiradas,CartasPC, CartasDescartas, baralho , cartaspossiveis]
        
        
        #------------------------4ªaposta-----------------------------------------------------
        
        # input do Aposta:ronda,SmallBlind, BigBlind, all_in, pote, JogadorBigBlind,dinheiroPC,dinheiroJogador,maoPC,pesosValor,historicoJogador,CartasRetiradas
        # output do Aposta:proximaronda, all_in, JogadorBigBlind,fold,pote,dinheiroPC,dinheiroJogador,historicoJogador
        a4 = Aposta(a3[0],SmallBlind,BigBlind,a3[1],a3[4],a3[2],a3[5],a3[6],out3[3],pesosValor,a3[7],CartasRetiradas,out3[0])
        print("\n")
        print("Valor do pote: ",a4[4])
        print("\n")

        if(a4[3]==1):
          return [a4[5],a4[6],out3[3],out3[0]]
          #output: [dinheiroPC,dinheiroJogador,maoPC,maoJogador]
        else:
          #showDown(maoJogador, maoPC, dinheiroPote, dinheiroJogador, dinheiroPC)
          r=showDown(out3[0],out3[3],a4[4],a4[6],a4[5])
          return [r[0],r[1],out3[3],out3[0]]
          #output: [dinheiroPC,dinheiroJogador,maoPC,maoJogador]

O Jogo começa aqui

In [6]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import random
import re
from itertools import chain,repeat,count,islice
from collections import Counter
import itertools

################################################## FUNÇÃO START ##############################################################

def start():
  #Definir a small e a big blind
  print("####################################################")
  print("############### Bem vindo ao Badugi! ###############")
  print("####################################################")
  print("\n\n")
  SmallBlind = 50
  BigBlind = 100
  
  print("Valor da BigBlind: ",BigBlind)
  print("Valor da SmallBlind: ",SmallBlind)

  print("\n")
  #Definir o dinheiro do PC e Jogador
  dinheiroPC = 5000
  dinheiroJogador = 5000
  d = [dinheiroPC,dinheiroJogador]

  print("Dinheiro do Jogador: ",dinheiroJogador)
  print("Dinheiro do PC: ",dinheiroPC)

  print("\n")

  #ver quem vai ter a big blind 
  JogadorBigBlind = random.randrange(2)#da-nos 0(Pc com big blind) ou 1(jogador com big blind)
  
  #esta variavel vai-nos dizer se o jogador quer continuar a jogar 
  continuar = 1

  #Nota: Como não se encontrou nenhuma regra que dita o que acontece quando um jogador, no inicio do jogo, tem o mesmo ou mais dinheiro que a small blind e menos que a big,
  #... vamos considerar: Caso um jogador, no inicio do jogo, tenha menos dinheiro que a big blind, perde automaticamente.   

  contaJogo = 1

  while(continuar and (not(JogadorBigBlind) or  JogadorBigBlind) and (d[0]>=BigBlind) and (d[1]>=BigBlind)  ):
    #o jogo continua se:
    #   o jogador quer continuar e tem dinheiro>=BigBlind
    #   o pc tem dinheiro>=BigBlind
    print("##################################################\n##################### Jogo " + str(contaJogo) + " #####################\n##################################################\n")
    
    d = Jogo(dinheiroPC,dinheiroJogador,JogadorBigBlind,SmallBlind,BigBlind)
    #output do jogo: [dinheiroPC,dinheiroJogador,maoPC,maoJogador]
    dinheiroJogador=d[1]
    dinheiroPC=d[0]
    contaJogo+=1
    
    continuar=1000
    while(continuar>1 or continuar<0):
      print("Pretende continuar na mesa a jogar?\nDigite 1 - quer continuar\nDigite 0 - quer sair\n")
      x = input("Digite a sua opcao: ")
      continuar = int(x)

    #No proximo jogo o jogador que tem a big blind vai passar a ter a small blind
    JogadorBigBlind = int(not(JogadorBigBlind))

  
  if(d[0]>d[1]):
    #dinheiroPC > dinheiroJogador
  
    print("O computador saiu com mais dinheiro")
    print("Dinheiro do PC: ",d[0])
    print("Dinheiro do Jogador: ",d[1])
    print("Mão do computador: ",d[2])
    print("Mão do Jogador: ",d[3])
  elif(d[0]<d[1]):
    #dinheiroPC<dinheiroJogador
    print("Parabéns! Você saiu com mais dinheiro que o computador.")
    print("Dinheiro do PC: ",d[0])
    print("Dinheiro do Jogador: ",d[1])
    print("Mão do computador: ",d[2])
    print("Mão do Jogador: ",d[3])
  else:
    #O jogo ficou empatado
    print("Boa! Não perdeu dinheiro.")
    print("Dinheiro do PC: ",d[0])
    print("Dinheiro do Jogador: ",d[1])
    print("Mão do computador: ",d[2])
    print("Mão do Jogador: ",d[3])









#################### INICIO #######################

start()


####################################################
############### Bem vindo ao Badugi! ###############
####################################################



Valor da BigBlind:  100
Valor da SmallBlind:  50


Dinheiro do Jogador:  5000
Dinheiro do PC:  5000


##################################################
##################### Jogo 1 #####################
##################################################



O PC tem a Big Blind!
O Jogador tem a Small Blind!
Quem começa é o Jogador!


################# Ronda nº:1 ######################
É a vez do Jogador:
Mão do Jogador: ['8E', 'DE', '2P', '5O']

Pretende dar fold?
Se sim, digite 1
Se não, digite 0

0
O valor da sua aposta anterior é: 50
O valor da aposta do PC vai em: 100
Digite o valor inteiro que pretende acrescentar à sua aposta.
Se pretender dar check insira o valor 0.

Dinheiro disponivel:  4950
Aumento da aposta = 0
Resposta invalida!
Possiveis hipoteses:
1 - Nao tem dinheiro para pagar
2 - o aumento foi demasiado baixo
